In [6]:
#Lombardia

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyproj

# Create a SparkSession
spark = SparkSession.builder.appName("timeseries").config('spark.sql.shuffle.partitions',500).config('spark.driver.maxResultSize', '10G') .config("spark.driver.memory", "32g").config("spark.executor.memory", "16g").config("spark.task.maxFailures", "10").config("spark.executor.instances", "16").config("spark.local.dir", "/afs/enea.it/por/user/nafis/PFS/tmp").getOrCreate()


# Load the data from the CSV file

schema = StructType(
   [StructField('x', DoubleType(), True),
    StructField('y', DoubleType(), True),
    StructField('z', DoubleType(), True),
    StructField('time', TimestampType(), True),
    StructField('c_PM25', DoubleType(), True),
    StructField('c_PM10', DoubleType(), True),
    StructField('c_O3', DoubleType(), True),
    StructField('c_NO2', DoubleType(), True),
    StructField('geometry', StringType(), True),
    StructField('index_right', IntegerType(), True),
    StructField('OBJECTID', IntegerType(), True),
    StructField('COD_REG', IntegerType(), True),
    StructField('COD_PRO', IntegerType(), True),
    StructField('NOME', StringType(), True),
    StructField('SIGLA', StringType(), True),
    StructField('Shape_Leng', DoubleType(), True),
    StructField('Shape_Area', DoubleType(), True),
   ]
  )


df = spark.read.format("csv").load("/afs/enea.it/por/user/nafis/PFS/tmp/nafi/data_chunk/lombardia_prov/", header=True, schema=schema)

# Select the columns in the desired order
df = df.select( 'x', 'y', 'z', 'time', 'c_PM25', 'c_PM10', 'c_O3', 'c_NO2', 'geometry', 'NOME')


df = df.withColumn("x", df["x"].cast(DoubleType())) \
    .withColumn("y", df["y"].cast(DoubleType())) \
    .withColumn("z", df["z"].cast(DoubleType())) \
    .withColumn("time", df["time"].cast(TimestampType())) \
    .withColumn("c_PM25", df["c_PM25"].cast(DoubleType())) \
    .withColumn("c_PM10", df["c_PM10"].cast(DoubleType())) \
    .withColumn("c_O3", df["c_O3"].cast(DoubleType())) \
    .withColumn("c_NO2", df["c_NO2"].cast(DoubleType())) \
    .withColumn("geometry", df["geometry"].cast(StringType()))\
    .withColumn("NOME", df["NOME"].cast(StringType()))

df = df.withColumnRenamed("c_PM25", "c_PM2_5")

df.show()

df.printSchema()

+--------+---------+----+-------------------+------------------+------------------+------------------+------------------+--------------------+-----+
|       x|        y|   z|               time|           c_PM2_5|            c_PM10|              c_O3|             c_NO2|            geometry| NOME|
+--------+---------+----+-------------------+------------------+------------------+------------------+------------------+--------------------+-----+
|462000.0|5016000.0|20.0|2019-11-24 00:00:00|         1.6957362|         2.7321966|          50.69704| 4.208417400000001|POINT (462000 501...|Pavia|
|462000.0|5016000.0|20.0|2019-11-24 01:00:00|1.5195319999999999|         2.5212698|         52.217495|3.6308307999999996|POINT (462000 501...|Pavia|
|462000.0|5016000.0|20.0|2019-11-24 02:00:00|1.4698639999999998|          2.540143|53.528316000000004|         3.0628326|POINT (462000 501...|Pavia|
|462000.0|5016000.0|20.0|2019-11-24 03:00:00|         1.4075043|         2.5400522|         55.269703|    

In [7]:
from pyspark.sql import functions as F
import pandas as pd

data = df.withColumnRenamed("time", "date_time")


# Convert the "date_time" column to a datetime object
data = data.withColumn("date_time", F.to_timestamp("date_time"))

# Extract the date and time values and store them in separate columns
data = data.withColumn("date", F.date_trunc("day", "date_time"))
data = data.withColumn("time", F.date_trunc("hour", "date_time"))

data = data.withColumnRenamed("date_time", "original_date_time")

# data.show()

In [8]:
from pyspark.sql.functions import col, when

# Create the season column based on the month of the date_column
data = data.withColumn(
    "season",
    when(col("original_date_time").between("2019-01-01 00:00:00", "2019-03-19 23:00:00") | col("original_date_time").between("2019-12-21 00:00:00", "2020-03-19 23:00:00") | col("original_date_time").between("2020-12-21 00:00:00", "2021-03-19 23:00:00") | col("original_date_time").between("2021-12-21 00:00:00", "2022-03-19 23:00:00") | col("original_date_time").between("2022-12-21 00:00:00", "2023-03-19 23:00:00"), "Winter")
    .when(col("original_date_time").between("2019-03-20 00:00:00", "2019-06-20 23:00:00") | col("original_date_time").between("2020-03-20 00:00:00", "2020-06-20 23:00:00") | col("original_date_time").between("2021-03-20 00:00:00", "2021-06-20 23:00:00") | col("original_date_time").between("2022-03-20 00:00:00", "2022-06-20 23:00:00") | col("original_date_time").between("2023-03-20 00:00:00", "2023-06-20 23:00:00"), "Spring")
    .when(col("original_date_time").between("2019-06-21 00:00:00", "2019-09-21 23:00:00") | col("original_date_time").between("2020-06-21 00:00:00", "2020-09-21 23:00:00") | col("original_date_time").between("2021-06-21 00:00:00", "2021-09-21 23:00:00") | col("original_date_time").between("2022-06-21 00:00:00", "2022-09-21 23:00:00") | col("original_date_time").between("2023-06-21 00:00:00", "2023-09-21 23:00:00"), "Summer")
    .when(col("original_date_time").between("2019-09-22 00:00:00", "2019-12-20 23:00:00") | col("original_date_time").between("2020-09-22 00:00:00", "2020-12-20 23:00:00") | col("original_date_time").between("2021-09-22 00:00:00", "2021-12-20 23:00:00") | col("original_date_time").between("2022-09-22 00:00:00", "2022-12-20 23:00:00") | col("original_date_time").between("2023-09-22 00:00:00", "2023-12-20 23:00:00"), "Autumn")
)


# data.show()

In [9]:
dataset = data.select(col("c_PM2_5"), col("c_PM10"), col("c_O3"), col("c_NO2"))
dataset_stat = dataset.describe()
dataset_stat.show()

+-------+------------------+------------------+--------------------+------------------+
|summary|           c_PM2_5|            c_PM10|                c_O3|             c_NO2|
+-------+------------------+------------------+--------------------+------------------+
|  count|          46463400|          46463400|            46463400|          46463400|
|   mean|12.512844285265684|13.898096026376539|   61.34618794631837|14.096244421018927|
| stddev|13.768673804829088|14.597778711778924|  30.753991251264225|18.666454366512447|
|    min|       0.019527944|       0.020731749|9.802752999999999...|       0.007907552|
|    max|          564.8532|          589.7105|            544.2421|         337.89258|
+-------+------------------+------------------+--------------------+------------------+



In [4]:
data_PM25 = data.select(col("original_date_time"),col("c_PM2_5"), col("x"), col("y"), col("NOME"))
data_PM25.show()

+-------------------+------------------+--------+---------+-----+
| original_date_time|           c_PM2_5|       x|        y| NOME|
+-------------------+------------------+--------+---------+-----+
|2019-11-24 00:00:00|         1.6957362|462000.0|5016000.0|Pavia|
|2019-11-24 01:00:00|1.5195319999999999|462000.0|5016000.0|Pavia|
|2019-11-24 02:00:00|1.4698639999999998|462000.0|5016000.0|Pavia|
|2019-11-24 03:00:00|         1.4075043|462000.0|5016000.0|Pavia|
|2019-11-24 04:00:00|1.3243808000000001|462000.0|5016000.0|Pavia|
|2019-11-24 05:00:00|1.2917066000000001|462000.0|5016000.0|Pavia|
|2019-11-24 06:00:00|         1.3729354|462000.0|5016000.0|Pavia|
|2019-11-24 07:00:00|         1.8005681|462000.0|5016000.0|Pavia|
|2019-11-24 08:00:00|         2.7273355|462000.0|5016000.0|Pavia|
|2019-11-24 09:00:00|4.3827419999999995|462000.0|5016000.0|Pavia|
|2019-11-24 10:00:00| 4.251999400000001|462000.0|5016000.0|Pavia|
|2019-11-24 11:00:00|3.8547763999999995|462000.0|5016000.0|Pavia|
|2019-11-2

In [5]:
data_PM10 = data.select(col("original_date_time"),col("c_PM10"), col("x"), col("y"), col("NOME"))
data_PM10.show()

+-------------------+------------------+--------+---------+-----+
| original_date_time|            c_PM10|       x|        y| NOME|
+-------------------+------------------+--------+---------+-----+
|2019-11-24 00:00:00|         2.7321966|462000.0|5016000.0|Pavia|
|2019-11-24 01:00:00|         2.5212698|462000.0|5016000.0|Pavia|
|2019-11-24 02:00:00|          2.540143|462000.0|5016000.0|Pavia|
|2019-11-24 03:00:00|         2.5400522|462000.0|5016000.0|Pavia|
|2019-11-24 04:00:00|         2.4868004|462000.0|5016000.0|Pavia|
|2019-11-24 05:00:00|         2.4511726|462000.0|5016000.0|Pavia|
|2019-11-24 06:00:00|         2.5612066|462000.0|5016000.0|Pavia|
|2019-11-24 07:00:00|         2.9130769|462000.0|5016000.0|Pavia|
|2019-11-24 08:00:00|         3.6733246|462000.0|5016000.0|Pavia|
|2019-11-24 09:00:00|         5.3303866|462000.0|5016000.0|Pavia|
|2019-11-24 10:00:00|         5.3588486|462000.0|5016000.0|Pavia|
|2019-11-24 11:00:00|          4.963749|462000.0|5016000.0|Pavia|
|2019-11-2

In [6]:
data_NO2 = data.select(col("original_date_time"),col("c_NO2"), col("x"), col("y"), col("NOME"))
data_NO2.show()

+-------------------+------------------+--------+---------+-----+
| original_date_time|             c_NO2|       x|        y| NOME|
+-------------------+------------------+--------+---------+-----+
|2019-11-24 00:00:00| 4.208417400000001|462000.0|5016000.0|Pavia|
|2019-11-24 01:00:00|3.6308307999999996|462000.0|5016000.0|Pavia|
|2019-11-24 02:00:00|         3.0628326|462000.0|5016000.0|Pavia|
|2019-11-24 03:00:00|          2.444656|462000.0|5016000.0|Pavia|
|2019-11-24 04:00:00|         1.9309062|462000.0|5016000.0|Pavia|
|2019-11-24 05:00:00|         1.6992023|462000.0|5016000.0|Pavia|
|2019-11-24 06:00:00|1.5480975000000001|462000.0|5016000.0|Pavia|
|2019-11-24 07:00:00|         2.1196866|462000.0|5016000.0|Pavia|
|2019-11-24 08:00:00|         4.0332394|462000.0|5016000.0|Pavia|
|2019-11-24 09:00:00|         7.3524475|462000.0|5016000.0|Pavia|
|2019-11-24 10:00:00|          6.889144|462000.0|5016000.0|Pavia|
|2019-11-24 11:00:00|          6.416884|462000.0|5016000.0|Pavia|
|2019-11-2

In [7]:
data_O3 = data.select(col("original_date_time"),col("c_O3"), col("x"), col("y"), col("NOME"))
data_O3.show()

+-------------------+------------------+--------+---------+-----+
| original_date_time|              c_O3|       x|        y| NOME|
+-------------------+------------------+--------+---------+-----+
|2019-11-24 00:00:00|          50.69704|462000.0|5016000.0|Pavia|
|2019-11-24 01:00:00|         52.217495|462000.0|5016000.0|Pavia|
|2019-11-24 02:00:00|53.528316000000004|462000.0|5016000.0|Pavia|
|2019-11-24 03:00:00|         55.269703|462000.0|5016000.0|Pavia|
|2019-11-24 04:00:00| 56.09563000000001|462000.0|5016000.0|Pavia|
|2019-11-24 05:00:00|56.588367000000005|462000.0|5016000.0|Pavia|
|2019-11-24 06:00:00|          56.55035|462000.0|5016000.0|Pavia|
|2019-11-24 07:00:00|          53.39305|462000.0|5016000.0|Pavia|
|2019-11-24 08:00:00|         45.281013|462000.0|5016000.0|Pavia|
|2019-11-24 09:00:00|         39.011024|462000.0|5016000.0|Pavia|
|2019-11-24 10:00:00|         42.592384|462000.0|5016000.0|Pavia|
|2019-11-24 11:00:00|42.843109999999996|462000.0|5016000.0|Pavia|
|2019-11-2

# Prophet Model

In [8]:
import io, os, sys, setuptools, tokenize
from pyspark.sql.types import StructType,StructField,StringType,TimestampType,ArrayType,DoubleType
from pyspark.sql.functions import current_date
from pyspark.sql.functions import pandas_udf, PandasUDFType
import matplotlib.pyplot as plt
from prophet import Prophet
from datetime import datetime
import pandas as pd
from time import time

In [10]:
# select the "first_name" and "last_name" columns and get distinct values
unique_coordinates = data.select(col("x"), col("y"), col("NOME")).distinct()

# show the unique values
unique_coordinates.show(60)

+--------+---------+-------+
|       x|        y|   NOME|
+--------+---------+-------+
|554000.0|5068000.0|Bergamo|
|598000.0|5016000.0|Brescia|
|666000.0|4996000.0|Mantova|
|566000.0|5120000.0|Sondrio|
|546000.0|5040000.0|Bergamo|
|546000.0|5064000.0|Bergamo|
|546000.0|5076000.0|Bergamo|
|598000.0|5052000.0|Brescia|
|634000.0|4996000.0|Mantova|
|638000.0|4984000.0|Mantova|
|474000.0|5000000.0|  Pavia|
|486000.0|5004000.0|  Pavia|
|542000.0|5032000.0|Cremona|
|554000.0|5060000.0|Bergamo|
|534000.0|5132000.0|Sondrio|
|474000.0|5068000.0| Varese|
|626000.0|4996000.0|Mantova|
|518000.0|5004000.0|  Pavia|
|582000.0|5148000.0|Sondrio|
|554000.0|5080000.0|Bergamo|
|638000.0|4988000.0|Mantova|
|622000.0|5064000.0|Brescia|
|518000.0|4968000.0|  Pavia|
|490000.0|5080000.0| Varese|
|506000.0|5020000.0| Milano|
|602000.0|5060000.0|Brescia|
|562000.0|5080000.0|Bergamo|
|622000.0|5052000.0|Brescia|
|626000.0|4992000.0|Mantova|
|482000.0|5104000.0| Varese|
|526000.0|5020000.0| Milano|
|562000.0|5068

PM2.5

In [9]:
from pyspark.sql.functions import *
from time import time
import matplotlib.pyplot as plt
import os

data_PM25_prophet = data_PM25.groupBy("NOME", "original_date_time").agg(avg("c_PM2_5").alias("avg_c_PM25"))
data_PM25_prophet.show()

data_PM25_prophet_u = data_PM25_prophet.withColumnRenamed("avg_c_PM25", "y").withColumnRenamed("original_date_time", "ds")

data_PM25_prophet = data_PM25_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
                                       .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))

data_PM25_prophet.printSchema()

def train_and_forecast(group):
    
    print(f"Processing group: {group['NOME'].iloc[0]} with {len(group)} data points")
    
    # Initiate the model
    
    # Convert "ds" column to pandas datetime format and set timezone to Europe/Rome
    group["ds"] = pd.to_datetime(group["ds"])
    
    m = Prophet(seasonality_mode="additive",
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    interval_width=0.95)
    
    # Fit the model
    m.fit(group[group['ds'] <= cutoff_date])
    
    # Make predictions
    forecast = m.predict(group)

    # Create the directory if it does not exist
    path = '/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/PM25/'

    if not os.path.exists(path):
        os.makedirs(path)
    
    try:
        fig1 = m.plot(forecast)
        fig1.suptitle(f"PM2.5 Concentration Forecast for {group['NOME'].iloc[0]}")
        fig1.legend(['Actual', 'Forecasted'], loc='upper right')
        fig1.subplots_adjust(top=0.9)
        
        fig2 = m.plot_components(forecast)
        fig2.suptitle(f"PM2.5 Concentration Component Forecast for {group['NOME'].iloc[0]}")
        fig2.subplots_adjust(top=0.9)

        # Save the plots as images using Matplotlib's savefig method
        fig1.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/PM25/{group['NOME'].iloc[0]}_prophet_forecast.png")
        fig2.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/PM25/{group['NOME'].iloc[0]}_prophet_components.png")
    
    except Exception as e:
        print(f"Error generating plots for group {group['NOME'].iloc[0]}: {str(e)}")
    
    f_pd = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','trend', 'yearly', 'weekly', 'daily']].set_index("ds")
 
    group_pd = group[['ds', 'NOME', 'y']].set_index("ds")
    
    result_pd = f_pd.join( group_pd, how = "left")
    result_pd.reset_index(level=0, inplace=True)
    
    result_pd['NOME'] = group['NOME'].iloc[0]
    
    #Return the forecasted results
    return result_pd[['ds', 'NOME', 'y', 'yhat', 'yhat_upper', 'yhat_lower','trend', 'yearly', 'weekly', 'daily']] 

# Define the result schema
result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('NOME', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat',FloatType()),
    StructField('yhat_upper',FloatType()),
    StructField('yhat_lower',FloatType()),
    StructField('trend',FloatType()),
    StructField('yearly',FloatType()),
    StructField('weekly',FloatType()),
    StructField('daily',FloatType())
  ])

# create train and test data based on cutoff date
cutoff_date = "2021-06-30 23:00:00"

data_PM25_prophet_train = data_PM25_prophet.filter(col('ds') <= cutoff_date)
data_PM25_prophet_test = data_PM25_prophet.filter((col('ds') > cutoff_date))

max_date = data_PM25_prophet_test.agg(max('ds')).collect()[0][0]

period = data_PM25_prophet_test.count()

# Start time
start_time = time()
# Train and forecast by ticker 
spark_forecast_PM25 = data_PM25_prophet.groupBy("NOME").applyInPandas(train_and_forecast, schema=result_schema)
# Processing time
print('The time used for the Spark forecast is ', time()-start_time)

spark_forecast_PM25.show()


+-------+-------------------+------------------+
|   NOME| original_date_time|        avg_c_PM25|
+-------+-------------------+------------------+
|Sondrio|2019-11-24 14:00:00|0.4629849650753766|
| Varese|2020-08-04 06:00:00|1.8775407802631583|
| Varese|2021-07-11 17:00:00|1.6206620052631573|
|Brescia|2019-11-23 07:00:00|1.5688579178333328|
| Varese|2021-06-27 10:00:00| 5.575907214473685|
|  Pavia|2021-06-26 01:00:00| 2.940159353475935|
| Varese|2021-06-26 05:00:00|3.0987177657894738|
| Milano|2021-11-28 12:00:00| 11.00185946145833|
| Varese|2022-05-29 00:00:00|2.5673966217105266|
|   Como|2021-05-17 10:00:00| 2.460556795308643|
|Mantova|2019-11-23 22:00:00|2.7499121772413795|
|  Lecco|2022-06-11 19:00:00| 3.733884223529412|
|  Pavia|2022-06-11 03:00:00| 4.167420760962564|
| Varese|2021-06-26 15:00:00| 4.034924709210528|
|Sondrio|2019-11-24 04:00:00|0.3275477624623116|
|   Lodi|2020-08-04 21:00:00| 5.838125678431372|
|Brescia|2021-05-17 12:00:00| 3.178358130400001|
|  Lecco|2022-06-11 

The time used for the Spark forecast is  0.7806403636932373


Processing group: Pavia with 29862 data points                      (0 + 1) / 1]
06:00:18 - cmdstanpy - INFO - Chain [1] start processing
06:00:37 - cmdstanpy - INFO - Chain [1] done processing


+-------------------+-----+------------------+---------+----------+----------+---------+----------+-----------+-----------+
|                 ds| NOME|                 y|     yhat|yhat_upper|yhat_lower|    trend|    yearly|     weekly|      daily|
+-------------------+-----+------------------+---------+----------+----------+---------+----------+-----------+-----------+
|2019-01-01 01:00:00|Pavia|14.837051558288778|13.018844|   28.3833|-1.3219028|1.2515209| 10.278912|-0.23362213|  1.7220335|
|2019-01-01 02:00:00|Pavia| 14.85506893368985|13.009508| 27.529324|-1.2241693|1.2615663| 10.293717|-0.17368807|  1.6279119|
|2019-01-01 03:00:00|Pavia|15.610032396256681| 12.94114| 28.425047|-1.5791662|1.2716117| 10.308579|-0.11854631|  1.4794955|
|2019-01-01 04:00:00|Pavia|  16.8084777459893|12.830381| 27.490923|-2.4220767|1.2816572| 10.323495| -0.0681153|  1.2933443|
|2019-01-01 05:00:00|Pavia|15.544465164171127| 12.68904| 28.060505|-1.5802523|1.2917026| 10.338465| -0.0222586|  1.0811312|
|2019-01

Processing group: Varese with 29862 data points


In [12]:
from pyspark.sql.functions import col
from pyspark.sql.functions import pow, sqrt

# Calculate MAE
mae_PM25 = spark_forecast_PM25.select(col("NOME"), abs(col("y") - col("yhat")).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MAE")

# Calculate MSE
mse_PM25 = spark_forecast_PM25.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MSE")

# Calculate RMSE
rmse_PM25 = spark_forecast_PM25.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                     .groupBy("NOME") \
                     .agg({"error": "mean"}) \
                     .withColumnRenamed("avg(error)", "MSE") \
                     .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE"))

# Show the results
mae_PM25.show()
mse_PM25.show()
rmse_PM25.show()

23:22:50 - cmdstanpy - INFO - Chain [1] start processing        (40 + 40) / 135]
Processing group: Bergamo with 29862 data pointsProcessing group: Pavia with 29862 data points

Processing group: Brescia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Cremona with 29862 data points
23:23:01 - cmdstanpy - INFO - Chain [1] start processing
23:23:01 - cmdstanpy - INFO - Chain [1] start processing
23:23:01 - cmdstanpy - INFO - Chain [1] start processing
23:23:01 - cmdstanpy - INFO - Chain [1] start processing
23:23:01 - cmdstanpy - INFO - Chain [1] start processing
23:23:01 - cmdstanpy - INFO - Chain [1] start processing
23:23:10 - cmdstanpy - INFO - Chain [1] done processing
23:23:30 - cmdstanpy - INFO - Chain [1] done processing
23:23:34 - cmdstanpy - INFO - Chain [1] done processing
23:23:34 - cmdstanpy - INFO - Chain [1] done processing
23:23:38 - cmdstanpy - INFO - Chain [1] done processing
23:23:41 -

+--------------------+------------------+
|                NOME|               MAE|
+--------------------+------------------+
|               Pavia| 6.339380366983641|
|              Varese| 9.912417803316702|
|             Brescia| 6.718592697687178|
|             Sondrio|2.7768963125078767|
|Monza e della Bri...|17.673961351895326|
|                Lodi| 7.566810560961431|
|                Como|  8.32234208405971|
|             Bergamo| 7.702440946759207|
|              Milano|13.416569839343156|
|             Cremona| 7.760594695191041|
|             Mantova| 6.773280441750467|
|               Lecco| 9.160889824646938|
+--------------------+------------------+



Processing group: Brescia with 29862 data points==============> (130 + 5) / 135]
Processing group: Lecco with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Pavia with 29862 data points
23:24:56 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
23:24:56 - cmdstanpy - INFO - Chain [1] start processing
23:24:56 - cmdstanpy - INFO - Chain [1] start processing
23:24:56 - cmdstanpy - INFO - Chain [1] start processing
23:24:56 - cmdstanpy - INFO - Chain [1] start processing
23:24:56 - cmdstanpy - INFO - Chain [1] start processing
23:25:22 - cmdstanpy - INFO - Chain [1] done processing
23:25:23 - cmdstanpy - INFO - Chain [1] done processing
23:25:28 - cmdstanpy - INFO - Chain [1] done processing
23:25:32 - cmdstanpy - INFO - Chain [1] done processing
23:25:32 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Como with 29862 d

+--------------------+------------------+
|                NOME|               MSE|
+--------------------+------------------+
|               Pavia| 75.13884564769091|
|              Varese|176.30623582362665|
|             Brescia| 81.42756729191025|
|             Sondrio|14.038508989573522|
|Monza e della Bri...| 623.1304392511537|
|                Lodi|105.32005423394936|
|                Como|122.88365469240335|
|             Bergamo|107.75539226044744|
|              Milano| 327.4322101401099|
|             Cremona|107.46310742370433|
|             Mantova| 82.05080784267935|
|               Lecco|155.21670235306334|
+--------------------+------------------+



Processing group: Bergamo with 29862 data points==============> (132 + 3) / 135]
Processing group: Pavia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Cremona with 29862 data points
23:26:46 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
23:26:46 - cmdstanpy - INFO - Chain [1] start processing
23:26:46 - cmdstanpy - INFO - Chain [1] start processing
23:26:46 - cmdstanpy - INFO - Chain [1] start processing
23:26:46 - cmdstanpy - INFO - Chain [1] start processing
23:26:46 - cmdstanpy - INFO - Chain [1] start processing
23:27:13 - cmdstanpy - INFO - Chain [1] done processing
23:27:13 - cmdstanpy - INFO - Chain [1] done processing
23:27:18 - cmdstanpy - INFO - Chain [1] done processing
23:27:19 - cmdstanpy - INFO - Chain [1] done processing
23:27:20 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Como with 29862 d

+--------------------+------------------+------------------+
|                NOME|               MSE|              RMSE|
+--------------------+------------------+------------------+
|               Pavia| 75.13884564769091| 8.668266588406873|
|              Varese|176.30623582362665|13.278035842082467|
|             Brescia| 81.42756729191025| 9.023722474229261|
|             Sondrio|14.038508989573522|3.7467998331340735|
|Monza e della Bri...| 623.1304392511537| 24.96258078106416|
|                Lodi|105.32005423394936|10.262555930856083|
|                Como|122.88365469240335|11.085290013905967|
|             Bergamo|107.75539226044744|10.380529478810194|
|              Milano| 327.4322101401099|18.095088011394413|
|             Cremona|107.46310742370433|10.366441405984231|
|             Mantova| 82.05080784267935| 9.058190097512822|
|               Lecco|155.21670235306334|12.458599534179728|
+--------------------+------------------+------------------+



In [ ]:
#for long and short term forecast

from pyspark.sql.functions import *
from time import time
import matplotlib.pyplot as plt
import os

data_PM25_prophet = data_PM25.groupBy("NOME", "original_date_time").agg(avg("c_PM2_5").alias("avg_c_PM25"))
data_PM25_prophet.show()

data_PM25_prophet_u = data_PM25_prophet.withColumnRenamed("avg_c_PM25", "y").withColumnRenamed("original_date_time", "ds")

data_PM25_prophet = data_PM25_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
                                       .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))

data_PM25_prophet.printSchema()

def train_and_forecast(group):
    
    print(f"Processing group: {group['NOME'].iloc[0]} with {len(group)} data points")
    
    # Initiate the model
    
    # Convert "ds" column to pandas datetime format and set timezone to Europe/Rome
    group["ds"] = pd.to_datetime(group["ds"])
    
    m = Prophet(seasonality_mode="additive",
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    interval_width=0.95)
    
    # Fit the model
    m.fit(group[group['ds'] <= cutoff_date])
    
    # Make predictions
    forecast = m.predict(group)

    # Create the directory if it does not exist
    path = '/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/PM25/'

    if not os.path.exists(path):
        os.makedirs(path)
    
   # try:
    #    fig1 = m.plot(forecast)
     #   fig1.suptitle(f"PM2.5 Concentration Forecast for {group['NOME'].iloc[0]}")
     #   fig1.legend(['Actual', 'Forecasted'], loc='upper right')
     #   fig1.subplots_adjust(top=0.9)
        
      #  fig2 = m.plot_components(forecast)
       # fig2.suptitle(f"PM2.5 Concentration Component Forecast for {group['NOME'].iloc[0]}")
       # fig2.subplots_adjust(top=0.9)

        # Save the plots as images using Matplotlib's savefig method
        #fig1.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/PM25/{group['NOME'].iloc[0]}_prophet_forecast.png")
        #fig2.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/PM25/{group['NOME'].iloc[0]}_prophet_components.png")
    
    #except Exception as e:
     #   print(f"Error generating plots for group {group['NOME'].iloc[0]}: {str(e)}")
    
    f_pd = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','trend', 'yearly', 'weekly', 'daily']].set_index("ds")
 
    group_pd = group[['ds', 'NOME', 'y']].set_index("ds")
    
    result_pd = f_pd.join( group_pd, how = "left")
    result_pd.reset_index(level=0, inplace=True)
    
    result_pd['NOME'] = group['NOME'].iloc[0]
    
    #Return the forecasted results
    return result_pd[['ds', 'NOME', 'y', 'yhat', 'yhat_upper', 'yhat_lower','trend', 'yearly', 'weekly', 'daily']] 

# Define the result schema
result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('NOME', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat',FloatType()),
    StructField('yhat_upper',FloatType()),
    StructField('yhat_lower',FloatType()),
    StructField('trend',FloatType()),
    StructField('yearly',FloatType()),
    StructField('weekly',FloatType()),
    StructField('daily',FloatType())
  ])

# create train and test data based on cutoff date
cutoff_date = "2021-06-30 23:00:00"

data_PM25_prophet_train = data_PM25_prophet.filter(col('ds') <= cutoff_date)
data_PM25_prophet_test = data_PM25_prophet.filter((col('ds') > cutoff_date))

max_date = data_PM25_prophet_test.agg(max('ds')).collect()[0][0]

period = data_PM25_prophet_test.count()

# Start time
start_time = time()
# Train and forecast by ticker 
spark_forecast_PM25 = data_PM25_prophet.groupBy("NOME").applyInPandas(train_and_forecast, schema=result_schema)
# Processing time
print('The time used for the Spark forecast is ', time()-start_time)

spark_forecast_PM25.show()


In [11]:
from pyspark.sql.functions import col, pow, sqrt

# Define cutoff dates
cutoff_dates = {
    "5 Days": "2021-07-05 23:00:00",
    "30 Days": "2021-07-30 23:00:00",
    "90 Days": "2021-09-28 23:00:00",
    "180 Days": "2021-12-27 23:00:00",
    "365 Days": "2022-06-29 23:00:00"
}

# Define a list to store the evaluation results
results = []

# Compute evaluation metrics for each time period and NOME
for period, cutoff in cutoff_dates.items():
    for nome in spark_forecast_PM25.select("NOME").distinct().rdd.flatMap(lambda x: x).collect():
        # Filter the forecast data based on the cutoff date and NOME
        forecast_data_PM25 = spark_forecast_PM25.filter((col("ds") <= cutoff) & (col("NOME") == nome))

        # Calculate MAE
        mae = forecast_data_PM25.select(abs(col("y") - col("yhat")).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MAE") \
                             .collect()[0]["MAE"]

        # Calculate MSE
        mse = forecast_data_PM25.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MSE") \
                             .collect()[0]["MSE"]

        # Calculate RMSE
        rmse = forecast_data_PM25.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                              .agg({"error": "mean"}) \
                              .withColumnRenamed("avg(error)", "MSE") \
                              .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE")) \
                              .collect()[0]["RMSE"]

        # Calculate coverage
        coverage = forecast_data_PM25.filter((col("y") >= col("yhat_lower")) & (col("y") <= col("yhat_upper"))) \
                                  .count() / forecast_data_PM25.count() * 100

        # Add the results to the list
        results.append((period, nome, mse, rmse, mae, coverage))

# Create a Spark DataFrame from the results list
result_df_PM25 = spark.createDataFrame(results, ["Time", "NOME", "MSE", "RMSE", "MAE", "Coverage"])

# Show the results
result_df_PM25.show(100)

Processing group: Brescia with 29862 data points==>            (101 + 34) / 135]
Processing group: Lecco with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Cremona with 29862 data points
15:10:09 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
15:10:09 - cmdstanpy - INFO - Chain [1] start processing
15:10:09 - cmdstanpy - INFO - Chain [1] start processing
15:10:09 - cmdstanpy - INFO - Chain [1] start processing
15:10:09 - cmdstanpy - INFO - Chain [1] start processing
15:10:09 - cmdstanpy - INFO - Chain [1] start processing
15:10:39 - cmdstanpy - INFO - Chain [1] done processing
15:10:40 - cmdstanpy - INFO - Chain [1] done processing
15:10:40 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Milano with 29862 data points
15:10:48 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Sondrio with 29862 data p

Processing group: Lecco with 29862 data points============>    (122 + 13) / 135]
Processing group: Cremona with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Brescia with 29862 data points
15:19:19 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
15:19:19 - cmdstanpy - INFO - Chain [1] start processing
15:19:19 - cmdstanpy - INFO - Chain [1] start processing
15:19:19 - cmdstanpy - INFO - Chain [1] start processing
15:19:19 - cmdstanpy - INFO - Chain [1] start processing
15:19:20 - cmdstanpy - INFO - Chain [1] start processing
15:19:47 - cmdstanpy - INFO - Chain [1] done processing
15:19:48 - cmdstanpy - INFO - Chain [1] done processing
15:19:53 - cmdstanpy - INFO - Chain [1] done processing
15:19:54 - cmdstanpy - INFO - Chain [1] done processing
15:19:55 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Como with 29862

15:27:03 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
Processing group: Lodi with 29862 data points===========>      (117 + 18) / 135]
Processing group: Cremona with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Bergamo with 29862 data points
15:27:53 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
15:27:53 - cmdstanpy - INFO - Chain [1] start processing
15:27:53 - cmdstanpy - INFO - Chain [1] start processing
15:27:53 - cmdstanpy - INFO - Chain [1] start processing
15:27:53 - cmdstanpy - INFO - Chain [1] start processing
15:27:53 - cmdstanpy - INFO - Chain [1] start processing
15:28:24 - cmdstanpy - INFO - Chain [1] done processing
15:28:24 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Como with 29862 data points
15:28:32 - cmdstanpy - INFO - Chain [1] done processing
Processing group:

15:35:50 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
15:35:58 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Cremona with 29862 data points=========>     (120 + 15) / 135]
Processing group: Pavia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Brescia with 29862 data points
15:36:45 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
15:36:45 - cmdstanpy - INFO - Chain [1] start processing
15:36:45 - cmdstanpy - INFO - Chain [1] start processing
15:36:45 - cmdstanpy - INFO - Chain [1] start processing
15:36:45 - cmdstanpy - INFO - Chain [1] start processing
15:36:45 - cmdstanpy - INFO - Chain [1] start processing
15:37:15 - cmdstanpy - INFO - Chain [1] done processing
15:37:16 - cmdstanpy - INFO - Chain [1] done processing
15:37:22 - cmdstanpy - INFO - Chain [1] d

15:44:41 - cmdstanpy - INFO - Chain [1] done processing
15:44:46 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
15:44:47 - cmdstanpy - INFO - Chain [1] done processing
15:44:48 - cmdstanpy - INFO - Chain [1] done processing
15:44:53 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
Processing group: Pavia with 29862 data points======>          (107 + 28) / 135]
Processing group: Lecco with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Bergamo with 29862 data points
15:45:43 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
15:45:43 - cmdstanpy - INFO - Chain [1] start processing
15:45:43 - cmdstanpy - INFO - Chain [1] start processing
15:45:43 - cmdstanpy - INFO - Chain [1] start processing
15:45:43 - cmdstanpy - INFO - Chain [1] start processing
15:45:43 - cmdstanpy - INFO - Chain [1] 

...

02:20:24 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Monza e della Brianza with 29862 data points      (0 + 6) / 6]
02:20:33 - cmdstanpy - INFO - Chain [1] start processing
02:20:46 - cmdstanpy - INFO - Chain [1] done processing
02:20:53 - cmdstanpy - INFO - Chain [1] done processing
02:20:57 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
02:20:57 - cmdstanpy - INFO - Chain [1] done processing
02:21:04 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
02:21:09 - cmdstanpy - INFO - Chain [1] done processing             (4 + 2) / 6]
Processing group: Brescia with 29862 data points======>        (112 + 23) / 135]
Processing group: Bergamo with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Lodi with 29862 data points
02:21:26 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6)

02:28:08 - cmdstanpy - INFO - Chain [1] start processing
02:28:09 - cmdstanpy - INFO - Chain [1] start processing
02:28:10 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Monza e della Brianza with 29862 data points      (0 + 6) / 6]
02:28:18 - cmdstanpy - INFO - Chain [1] start processing
02:28:36 - cmdstanpy - INFO - Chain [1] done processing
02:28:39 - cmdstanpy - INFO - Chain [1] done processing
02:28:41 - cmdstanpy - INFO - Chain [1] done processing
02:28:42 - cmdstanpy - INFO - Chain [1] done processing
02:28:48 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
02:28:52 - cmdstanpy - INFO - Chain [1] done processing             (4 + 2) / 6]
Processing group: Lodi with 29862 data points=======>          (108 + 27) / 135]
Processing group: Pavia with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Cremona with 2

23/05/19 02:32:56 WARN PythonWorkerFactory: Failed to open socket to Python daemon:
java.net.SocketException: Broken pipe (Write failed)
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:111)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:134)
	at java.io.DataOutputStream.writeInt(DataOutputStream.java:198)
	at org.apache.spark.security.SocketAuthHelper.writeUtf8(SocketAuthHelper.scala:112)
	at org.apache.spark.security.SocketAuthHelper.authToServer(SocketAuthHelper.scala:86)
	at org.apache.spark.api.python.PythonWorkerFactory.createSocket$1(PythonWorkerFactory.scala:127)
	at org.apache.spark.api.python.PythonWorkerFactory.liftedTree1$1(PythonWorkerFactory.scala:138)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:137)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPy

23/05/19 02:33:03 ERROR Executor: Exception in task 28.0 in stage 6130.0 (TID 95820)
org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:599)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:581)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:770)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:747)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.s

Py4JJavaError: An error occurred while calling o8109.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 28 in stage 6130.0 failed 1 times, most recent failure: Lost task 28.0 in stage 6130.0 (TID 95820) (cresco6-nvi1.portici.enea.it executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:599)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:581)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:770)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:747)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.

In [13]:
result_df_PM25.show(100)

+--------+--------------------+------------------+------------------+------------------+-----------------+
|    Time|                NOME|               MSE|              RMSE|               MAE|         Coverage|
+--------+--------------------+------------------+------------------+------------------+-----------------+
|  5 Days|               Pavia| 60.68464734318503| 7.790035131062313| 5.520306672223819| 94.3330223880597|
|  5 Days|              Varese|131.50498493452778|11.467562292594176|  8.44641439469029|93.98320895522389|
|  5 Days|             Brescia|46.951173500340126| 6.852092636584835| 5.092851209323915|94.23973880597015|
|  5 Days|             Sondrio| 10.46504829789024|3.2349726888940253| 2.319746067681182| 94.7901119402985|
|  5 Days|Monza e della Bri...| 281.3064144573052| 16.77219170106594|11.892615632389754| 94.0205223880597|
|  5 Days|                Lodi| 82.54195466050516| 9.085260296794207| 6.497650839857919|94.69682835820895|
|  5 Days|                Como| 89.29

In [ ]:
result_df_PM25_pd = result_df_PM25.pandas_api()
result_df_PM25_pd

PM10

In [14]:
from pyspark.sql.functions import *
from time import time
import matplotlib.pyplot as plt
import os

data_PM10_prophet = data_PM10.groupBy("NOME", "original_date_time").agg(avg("c_PM10").alias("avg_c_PM10"))
data_PM10_prophet.show()

data_PM10_prophet_u = data_PM10_prophet.withColumnRenamed("avg_c_PM10", "y").withColumnRenamed("original_date_time", "ds")
# data_PM25_prophet = data_PM25_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss"))
data_PM10_prophet = data_PM10_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
                                       .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))

data_PM10_prophet.printSchema()


def train_and_forecast(group):
    
    print(f"Processing group: {group['NOME'].iloc[0]} with {len(group)} data points")
    
    # Initiate the model
    
    # Convert "ds" column to pandas datetime format and set timezone to Europe/Rome
    #group["ds"] = pd.to_datetime(group["ds"]).dt.tz_localize('Europe/Rome').dt.tz_convert(pytz.UTC)
    group["ds"] = pd.to_datetime(group["ds"])

    m = Prophet(seasonality_mode="additive",
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    interval_width=0.95)
    #m.add_country_holidays(country_name="IT")
    
    # Fit the model
    m.fit(group[group['ds'] <= cutoff_date])
    
    
    # Make predictions
    forecast = m.predict(group)

#     future_forecast = m.predict(m.make_future_dataframe(periods=period, freq="H", include_history=True).loc[group['ds'] > cutoff_date])

    # Create the directory if it does not exist
    path = '/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/PM10/'

    if not os.path.exists(path):
        os.makedirs(path)
    
    try:
        fig1 = m.plot(forecast)
        fig1.suptitle(f"PM10 Concentration Forecast for {group['NOME'].iloc[0]}")
        fig1.legend(['Actual', 'Forecasted'], loc='upper right')
        fig1.subplots_adjust(top=0.9)
        
        fig2 = m.plot_components(forecast)
        fig2.suptitle(f"PM10 Concentration Component Forecast for {group['NOME'].iloc[0]}")
        fig2.subplots_adjust(top=0.9)

        # Save the plots as images using Matplotlib's savefig method
        fig1.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/PM10/{group['NOME'].iloc[0]}_prophet_forecast.png")
        fig2.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/PM10/{group['NOME'].iloc[0]}_prophet_components.png")
    except Exception as e:
        print(f"Error generating plots for group {group['NOME'].iloc[0]}: {str(e)}")
    
    
    f_pd = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','trend', 'yearly', 'weekly', 'daily']].set_index("ds")
 
    group_pd = group[['ds', 'NOME', 'y']].set_index("ds")
    
    result_pd = f_pd.join( group_pd, how = "left")
    result_pd.reset_index(level=0, inplace=True)
    
    result_pd['NOME'] = group['NOME'].iloc[0]
    
    #Return the forecasted results
    return result_pd[['ds', 'NOME', 'y', 'yhat', 'yhat_upper', 'yhat_lower','trend', 'yearly', 'weekly', 'daily']] 

# Define the restult schema
result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('NOME', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat',FloatType()),
    StructField('yhat_upper',FloatType()),
    StructField('yhat_lower',FloatType()),
    StructField('trend',FloatType()),
    StructField('yearly',FloatType()),
    StructField('weekly',FloatType()),
    StructField('daily',FloatType())
  ])

# create train and test data based on cutoff date
cutoff_date = "2021-06-30 23:00:00"

data_PM10_prophet_train = data_PM10_prophet.filter(col('ds') <= cutoff_date)
data_PM10_prophet_test = data_PM10_prophet.filter((col('ds') > cutoff_date))

max_date = data_PM10_prophet_test.agg(max('ds')).collect()[0][0]

period = data_PM10_prophet_test.count()

# Start time
start_time = time()
# Train and forecast by ticker 
spark_forecast_PM10 = data_PM10_prophet.groupBy("NOME").applyInPandas(train_and_forecast, schema=result_schema)
# Processing time
print('The time used for the Spark forecast is ', time()-start_time)

spark_forecast_PM10.show()

+-------+-------------------+------------------+
|   NOME| original_date_time|        avg_c_PM10|
+-------+-------------------+------------------+
|Sondrio|2019-11-24 14:00:00| 1.532554649246231|
| Varese|2020-08-04 06:00:00|2.2014404381578947|
| Varese|2021-07-11 17:00:00|1.7296184092105258|
|Brescia|2019-11-23 07:00:00|1.8590820483666675|
| Varese|2021-06-27 10:00:00|6.2345089894736825|
|  Pavia|2021-06-26 01:00:00| 4.496175512834227|
| Varese|2021-06-26 05:00:00|3.5258748842105265|
| Milano|2021-11-28 12:00:00|11.555928287500002|
| Varese|2022-05-29 00:00:00|2.8523651335526323|
|   Como|2021-05-17 10:00:00|3.9270444765432106|
|Mantova|2019-11-23 22:00:00| 4.798481184827586|
|  Lecco|2022-06-11 19:00:00| 4.521177043137254|
|  Pavia|2022-06-11 03:00:00|  4.70305849144385|
| Varese|2021-06-26 15:00:00| 4.952178510526315|
|Sondrio|2019-11-24 04:00:00|1.0391447534673364|
|   Lodi|2020-08-04 21:00:00| 8.042946805882352|
|Brescia|2021-05-17 12:00:00| 3.773742657633331|
|  Lecco|2022-06-11 

The time used for the Spark forecast is  0.04570198059082031


Processing group: Pavia with 29862 data points                      (0 + 1) / 1]
05:51:50 - cmdstanpy - INFO - Chain [1] start processing
05:52:18 - cmdstanpy - INFO - Chain [1] done processing


+-------------------+-----+------------------+---------+----------+-----------+---------+----------+------------+-----------+
|                 ds| NOME|                 y|     yhat|yhat_upper| yhat_lower|    trend|    yearly|      weekly|      daily|
+-------------------+-----+------------------+---------+----------+-----------+---------+----------+------------+-----------+
|2019-01-01 01:00:00|Pavia|15.880247914973262|14.435029| 30.474674|-0.53591835| 2.567699| 10.168847|-0.061331343|   1.759815|
|2019-01-01 02:00:00|Pavia|15.930497365240644|14.395037| 29.552923| -0.4641363|2.5774233| 10.183791|0.0044646943|  1.6293576|
|2019-01-01 03:00:00|Pavia| 16.74410628288769|14.307824| 30.472168| -0.8505734| 2.587148| 10.198796| 0.064097464|  1.4577827|
|2019-01-01 04:00:00|Pavia|17.886662908021396|14.167597| 29.472393| -1.7551272|2.5968723| 10.213861|  0.11766621|  1.2391964|
|2019-01-01 05:00:00|Pavia| 16.49189000695186|13.984689|  30.03165| -0.9116645|2.6065967| 10.228987|  0.16533734|  0.9

Processing group: Varese with 29862 data points


In [14]:
from pyspark.sql.functions import col
from pyspark.sql.functions import pow, sqrt
from prophet.diagnostics import performance_metrics


# Calculate MAE
mae_PM10 = spark_forecast_PM10.select(col("NOME"), abs(col("y") - col("yhat")).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MAE")


# Calculate MSE
mse_PM10 = spark_forecast_PM10.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MSE")

# Calculate RMSE
rmse_PM10 = spark_forecast_PM10.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                     .groupBy("NOME") \
                     .agg({"error": "mean"}) \
                     .withColumnRenamed("avg(error)", "MSE") \
                     .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE"))

# Show the results
mae_PM10.show()
mse_PM10.show()
rmse_PM10.show()

23:29:43 - cmdstanpy - INFO - Chain [1] start processing        (40 + 40) / 135]
Processing group: Bergamo with 29862 data points                    (0 + 6) / 6]
Processing group: Cremona with 29862 data points
Processing group: Pavia with 29862 data pointsProcessing group: Lodi with 29862 data points

Processing group: Lecco with 29862 data points
Processing group: Brescia with 29862 data points
23:29:58 - cmdstanpy - INFO - Chain [1] start processing
23:29:58 - cmdstanpy - INFO - Chain [1] start processing
23:29:58 - cmdstanpy - INFO - Chain [1] start processing
23:29:58 - cmdstanpy - INFO - Chain [1] start processing
23:29:58 - cmdstanpy - INFO - Chain [1] start processing
23:29:58 - cmdstanpy - INFO - Chain [1] start processing
23:30:07 - cmdstanpy - INFO - Chain [1] done processing
23:30:23 - cmdstanpy - INFO - Chain [1] done processing
23:30:24 - cmdstanpy - INFO - Chain [1] done processing
23:30:25 - cmdstanpy - INFO - Chain [1] done processing
23:30:28 - cmdstanpy - INFO - Chai

+--------------------+------------------+
|                NOME|               MAE|
+--------------------+------------------+
|               Pavia| 6.575181510204667|
|              Varese| 9.921747106366416|
|             Brescia| 6.827282026308845|
|             Sondrio|2.5865951474437696|
|Monza e della Bri...| 17.81853364838621|
|                Lodi| 8.013071288452633|
|                Como| 8.276787950209508|
|             Bergamo| 7.319581203196842|
|              Milano|14.621485193198716|
|             Cremona|  8.09621244065119|
|             Mantova| 6.981487766608171|
|               Lecco| 9.261364302836038|
+--------------------+------------------+



Processing group: Pavia with 29862 data points=================>(134 + 1) / 135]
Processing group: Lodi with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Cremona with 29862 data points
23:31:45 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
23:31:46 - cmdstanpy - INFO - Chain [1] start processing
23:31:46 - cmdstanpy - INFO - Chain [1] start processing23:31:46 - cmdstanpy - INFO - Chain [1] start processing

23:31:46 - cmdstanpy - INFO - Chain [1] start processing
23:31:46 - cmdstanpy - INFO - Chain [1] start processing
23:32:12 - cmdstanpy - INFO - Chain [1] done processing
23:32:13 - cmdstanpy - INFO - Chain [1] done processing
23:32:13 - cmdstanpy - INFO - Chain [1] done processing
23:32:15 - cmdstanpy - INFO - Chain [1] done processing
23:32:18 - cmdstanpy - INFO - Chain [1] done processing
23:32:19 - cmdstanpy - INFO - Cha

+--------------------+------------------+
|                NOME|               MSE|
+--------------------+------------------+
|               Pavia| 79.12658379144854|
|              Varese|174.97101232156894|
|             Brescia| 82.14205053353952|
|             Sondrio| 15.47768044571585|
|Monza e della Bri...| 612.2353089127952|
|                Lodi|115.10928945320863|
|                Como|120.19973912786452|
|             Bergamo| 93.74152432444878|
|              Milano|389.59927762334917|
|             Cremona|114.61170062065652|
|             Mantova| 85.42767102134144|
|               Lecco|155.64404796543334|
+--------------------+------------------+



Processing group: Bergamo with 29862 data points===============>(134 + 1) / 135]
Processing group: Brescia with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Lodi with 29862 data points
23:33:33 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
23:33:33 - cmdstanpy - INFO - Chain [1] start processing
23:33:33 - cmdstanpy - INFO - Chain [1] start processing
23:33:33 - cmdstanpy - INFO - Chain [1] start processing
23:33:33 - cmdstanpy - INFO - Chain [1] start processing
23:33:33 - cmdstanpy - INFO - Chain [1] start processing
23:33:56 - cmdstanpy - INFO - Chain [1] done processing
23:33:58 - cmdstanpy - INFO - Chain [1] done processing
23:34:00 - cmdstanpy - INFO - Chain [1] done processing
23:34:02 - cmdstanpy - INFO - Chain [1] done processing
23:34:03 - cmdstanpy - INFO - Chain [1] done processing
23:34:04 - cmdstanpy - INFO - Chain

+--------------------+------------------+------------------+
|                NOME|               MSE|              RMSE|
+--------------------+------------------+------------------+
|               Pavia| 79.12658379144854| 8.895312461709738|
|              Varese|174.97101232156894|13.227660878687846|
|             Brescia| 82.14205053353952| 9.063225172836628|
|             Sondrio| 15.47768044571585|3.9341683296112087|
|Monza e della Bri...| 612.2353089127952|24.743389196162987|
|                Lodi|115.10928945320863|10.728899731715673|
|                Como|120.19973912786452|10.963564161706927|
|             Bergamo| 93.74152432444878| 9.682020673622256|
|              Milano|389.59927762334917|19.738269367483795|
|             Cremona|114.61170062065652|10.705685434415516|
|             Mantova| 85.42767102134144| 9.242709073715424|
|               Lecco|155.64404796543334|12.475738373556627|
+--------------------+------------------+------------------+



In [ ]:
#for long and short term forecast
from pyspark.sql.functions import *
from time import time
import matplotlib.pyplot as plt
import os

data_PM10_prophet = data_PM10.groupBy("NOME", "original_date_time").agg(avg("c_PM10").alias("avg_c_PM10"))
data_PM10_prophet.show()

data_PM10_prophet_u = data_PM10_prophet.withColumnRenamed("avg_c_PM10", "y").withColumnRenamed("original_date_time", "ds")
# data_PM25_prophet = data_PM25_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss"))
data_PM10_prophet = data_PM10_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
                                       .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))

data_PM10_prophet.printSchema()


def train_and_forecast(group):
    
    print(f"Processing group: {group['NOME'].iloc[0]} with {len(group)} data points")
    
    # Initiate the model
    
    # Convert "ds" column to pandas datetime format and set timezone to Europe/Rome
    #group["ds"] = pd.to_datetime(group["ds"]).dt.tz_localize('Europe/Rome').dt.tz_convert(pytz.UTC)
    group["ds"] = pd.to_datetime(group["ds"])

    m = Prophet(seasonality_mode="additive",
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    interval_width=0.95)
    #m.add_country_holidays(country_name="IT")
    
    # Fit the model
    m.fit(group[group['ds'] <= cutoff_date])
    
    
    # Make predictions
    forecast = m.predict(group)

#     future_forecast = m.predict(m.make_future_dataframe(periods=period, freq="H", include_history=True).loc[group['ds'] > cutoff_date])

    # Create the directory if it does not exist
    path = '/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/PM10/'

    if not os.path.exists(path):
        os.makedirs(path)
    
    #try:
     #   fig1 = m.plot(forecast)
      #  fig1.suptitle(f"PM10 Concentration Forecast for {group['NOME'].iloc[0]}")
       # fig1.legend(['Actual', 'Forecasted'], loc='upper right')
       # fig1.subplots_adjust(top=0.9)
        
       # fig2 = m.plot_components(forecast)
       # fig2.suptitle(f"PM10 Concentration Component Forecast for {group['NOME'].iloc[0]}")
       # fig2.subplots_adjust(top=0.9)

        # Save the plots as images using Matplotlib's savefig method
        #fig1.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/PM10/{group['NOME'].iloc[0]}_prophet_forecast.png")
       # fig2.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/PM10/{group['NOME'].iloc[0]}_prophet_components.png")
   # except Exception as e:
   #     print(f"Error generating plots for group {group['NOME'].iloc[0]}: {str(e)}")
    
    
    f_pd = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','trend', 'yearly', 'weekly', 'daily']].set_index("ds")
 
    group_pd = group[['ds', 'NOME', 'y']].set_index("ds")
    
    result_pd = f_pd.join( group_pd, how = "left")
    result_pd.reset_index(level=0, inplace=True)
    
    result_pd['NOME'] = group['NOME'].iloc[0]
    
    #Return the forecasted results
    return result_pd[['ds', 'NOME', 'y', 'yhat', 'yhat_upper', 'yhat_lower','trend', 'yearly', 'weekly', 'daily']] 

# Define the restult schema
result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('NOME', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat',FloatType()),
    StructField('yhat_upper',FloatType()),
    StructField('yhat_lower',FloatType()),
    StructField('trend',FloatType()),
    StructField('yearly',FloatType()),
    StructField('weekly',FloatType()),
    StructField('daily',FloatType())
  ])

# create train and test data based on cutoff date
cutoff_date = "2021-06-30 23:00:00"

data_PM10_prophet_train = data_PM10_prophet.filter(col('ds') <= cutoff_date)
data_PM10_prophet_test = data_PM10_prophet.filter((col('ds') > cutoff_date))

max_date = data_PM10_prophet_test.agg(max('ds')).collect()[0][0]

period = data_PM10_prophet_test.count()

# Start time
start_time = time()
# Train and forecast by ticker 
spark_forecast_PM10 = data_PM10_prophet.groupBy("NOME").applyInPandas(train_and_forecast, schema=result_schema)
# Processing time
print('The time used for the Spark forecast is ', time()-start_time)

spark_forecast_PM10.show()

In [15]:
from pyspark.sql.functions import col, pow, sqrt

# Define cutoff dates
cutoff_dates = {
    "5 Days": "2021-07-05 23:00:00",
    "30 Days": "2021-07-30 23:00:00",
    "90 Days": "2021-09-28 23:00:00",
    "180 Days": "2021-12-27 23:00:00",
    "365 Days": "2022-06-29 23:00:00"
}

# Define a list to store the evaluation results
results = []

# Compute evaluation metrics for each time period and NOME
for period, cutoff in cutoff_dates.items():
    for nome in spark_forecast_PM10.select("NOME").distinct().rdd.flatMap(lambda x: x).collect():
        # Filter the forecast data based on the cutoff date and NOME
        forecast_data_PM10 = spark_forecast_PM10.filter((col("ds") <= cutoff) & (col("NOME") == nome))

        # Calculate MAE
        mae = forecast_data_PM10.select(abs(col("y") - col("yhat")).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MAE") \
                             .collect()[0]["MAE"]

        # Calculate MSE
        mse = forecast_data_PM10.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MSE") \
                             .collect()[0]["MSE"]

        # Calculate RMSE
        rmse = forecast_data_PM10.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                              .agg({"error": "mean"}) \
                              .withColumnRenamed("avg(error)", "MSE") \
                              .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE")) \
                              .collect()[0]["RMSE"]

        # Calculate coverage
        coverage = forecast_data_PM10.filter((col("y") >= col("yhat_lower")) & (col("y") <= col("yhat_upper"))) \
                                  .count() / forecast_data_PM10.count() * 100

        # Add the results to the list
        results.append((period, nome, mse, rmse, mae, coverage))

# Create a Spark DataFrame from the results list
result_df_PM10 = spark.createDataFrame(results, ["Time", "NOME", "MSE", "RMSE", "MAE", "Coverage"])

# Show the results
result_df_PM10.show(100)

05:52:30 - cmdstanpy - INFO - Chain [1] start processing         (0 + 96) / 135]
Processing group: Pavia with 29862 data points                      (0 + 6) / 6]
Processing group: Lecco with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Brescia with 29862 data points
05:53:11 - cmdstanpy - INFO - Chain [1] start processing
05:53:11 - cmdstanpy - INFO - Chain [1] start processing
05:53:11 - cmdstanpy - INFO - Chain [1] start processing
05:53:11 - cmdstanpy - INFO - Chain [1] start processing
05:53:11 - cmdstanpy - INFO - Chain [1] start processing
05:53:11 - cmdstanpy - INFO - Chain [1] start processing
05:53:19 - cmdstanpy - INFO - Chain [1] done processing
05:53:33 - cmdstanpy - INFO - Chain [1] done processing
05:53:37 - cmdstanpy - INFO - Chain [1] done processing
05:53:37 - cmdstanpy - INFO - Chain [1] done processing
05:53:40 - cmdstanpy - INFO - Ch

06:08:22 - cmdstanpy - INFO - Chain [1] done processing
06:08:31 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
06:08:34 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
06:08:34 - cmdstanpy - INFO - Chain [1] done processing
06:08:41 - cmdstanpy - INFO - Chain [1] done processing             (3 + 3) / 6]
Processing group: Pavia with 29862 data points=========>       (116 + 19) / 135]
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lodi with 29862 data points
06:09:26 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
06:09:26 - cmdstanpy - INFO - Chain [1] start processing
06:09:26 - cmdstanpy - INFO - Chain [1] start processing
06:09:26 - cmdstanpy - INFO - Chain [1] start processing
06:09:27 - cmdstanpy - INFO - Chain [1] start processing
06:09:27 - cmds

06:16:16 - cmdstanpy - INFO - Chain [1] start processing
06:16:26 - cmdstanpy - INFO - Chain [1] done processing
06:16:30 - cmdstanpy - INFO - Chain [1] done processing
06:16:38 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
06:16:39 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
06:16:41 - cmdstanpy - INFO - Chain [1] done processing
06:16:45 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Bergamo with 29862 data points========>      (119 + 16) / 135]
Processing group: Pavia with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lecco with 29862 data points
06:17:31 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
06:17:31 - cmdstanpy - INFO - Chain [1] start processing
06:17:31 - cmdstanpy - INFO - Chain [1] start processing
06:17:31 - cmdstanpy - INFO - Chain [1] sta

06:24:20 - cmdstanpy - INFO - Chain [1] start processing
06:24:34 - cmdstanpy - INFO - Chain [1] done processing
06:24:36 - cmdstanpy - INFO - Chain [1] done processing
06:24:40 - cmdstanpy - INFO - Chain [1] done processing             (0 + 6) / 6]
06:24:44 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
06:24:47 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
06:24:49 - cmdstanpy - INFO - Chain [1] done processing             (3 + 3) / 6]
Processing group: Brescia with 29862 data points====>          (107 + 28) / 135]
Processing group: Bergamo with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Lodi with 29862 data points
06:25:32 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
06:25:32 - cmdstanpy - INFO - Chain [1] start processing
06:25:32 - cmdstanpy - INFO - Chain [1] start proc

06:32:18 - cmdstanpy - INFO - Chain [1] start processing
06:32:18 - cmdstanpy - INFO - Chain [1] start processing
06:32:29 - cmdstanpy - INFO - Chain [1] done processing             (0 + 6) / 6]
06:32:30 - cmdstanpy - INFO - Chain [1] done processing
06:32:40 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
06:32:41 - cmdstanpy - INFO - Chain [1] done processing
06:32:42 - cmdstanpy - INFO - Chain [1] done processing
06:32:48 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Lodi with 29862 data points======>           (104 + 31) / 135]
Processing group: Cremona with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
06:33:34 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
06:33:34 - cmdstanpy - INFO - Chain [1] start processing
06:33:34 - cmdstanpy - INFO - Chain [1] 

...

14:52:36 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
14:52:36 - cmdstanpy - INFO - Chain [1] start processing
14:52:36 - cmdstanpy - INFO - Chain [1] start processing
14:52:36 - cmdstanpy - INFO - Chain [1] start processing
14:52:36 - cmdstanpy - INFO - Chain [1] start processing
14:52:36 - cmdstanpy - INFO - Chain [1] start processing
14:52:59 - cmdstanpy - INFO - Chain [1] done processing
14:53:00 - cmdstanpy - INFO - Chain [1] done processing
14:53:01 - cmdstanpy - INFO - Chain [1] done processing
14:53:03 - cmdstanpy - INFO - Chain [1] done processing
14:53:03 - cmdstanpy - INFO - Chain [1] done processing
14:53:03 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
Processing group: Milano with 29862 data points
14:53:09 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Como with 29862 data points
Processing group: Mantova with 29862 data points
14:53:11 - cmdstanpy - INFO - Chain

15:00:27 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
15:00:27 - cmdstanpy - INFO - Chain [1] start processing
15:00:27 - cmdstanpy - INFO - Chain [1] start processing
15:00:27 - cmdstanpy - INFO - Chain [1] start processing
15:00:27 - cmdstanpy - INFO - Chain [1] start processing
15:00:27 - cmdstanpy - INFO - Chain [1] start processing
15:00:49 - cmdstanpy - INFO - Chain [1] done processing
15:00:50 - cmdstanpy - INFO - Chain [1] done processing
15:00:52 - cmdstanpy - INFO - Chain [1] done processing
15:00:55 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
15:00:58 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Como with 29862 data points
15:00:59 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Milano with 29862 data points
15:01:00 - cmdstanpy - INFO - Chain [1] done processing
15:01:00 - cmdstanpy - INFO - Chain [1] start processing
15:01:02 - cmdstanpy - INFO

15:16:16 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
15:16:16 - cmdstanpy - INFO - Chain [1] start processing
15:16:16 - cmdstanpy - INFO - Chain [1] start processing
15:16:16 - cmdstanpy - INFO - Chain [1] start processing
15:16:16 - cmdstanpy - INFO - Chain [1] start processing
15:16:16 - cmdstanpy - INFO - Chain [1] start processing
15:16:44 - cmdstanpy - INFO - Chain [1] done processing
15:16:46 - cmdstanpy - INFO - Chain [1] done processing
15:16:48 - cmdstanpy - INFO - Chain [1] done processing
15:16:49 - cmdstanpy - INFO - Chain [1] done processing
15:16:51 - cmdstanpy - INFO - Chain [1] done processing
15:16:52 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
Processing group: Milano with 29862 data points
15:16:55 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Como with 29862 data points
15:16:57 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Sondrio w

15:24:18 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
15:24:18 - cmdstanpy - INFO - Chain [1] start processing
15:24:19 - cmdstanpy - INFO - Chain [1] start processing
15:24:19 - cmdstanpy - INFO - Chain [1] start processing
15:24:19 - cmdstanpy - INFO - Chain [1] start processing
15:24:19 - cmdstanpy - INFO - Chain [1] start processing
15:24:40 - cmdstanpy - INFO - Chain [1] done processing
15:24:45 - cmdstanpy - INFO - Chain [1] done processing
15:24:47 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Como with 29862 data points
15:24:51 - cmdstanpy - INFO - Chain [1] done processing
15:24:52 - cmdstanpy - INFO - Chain [1] start processing
15:24:52 - cmdstanpy - INFO - Chain [1] done processing
15:24:52 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
Processing group: Milano with 29862 data points
15:24:56 - cmdstanpy - INFO - Chain [1] start processing
15:24:58 - cmdstanpy - INFO

+--------+--------------------+------------------+------------------+------------------+-----------------+
|    Time|                NOME|               MSE|              RMSE|               MAE|         Coverage|
+--------+--------------------+------------------+------------------+------------------+-----------------+
|  5 Days|               Pavia| 66.11424676214602| 8.131066766553207| 5.865660484045035|94.39365671641791|
|  5 Days|              Varese|142.19520888013216|11.924563257416692| 8.871101386837646|93.99720149253731|
|  5 Days|             Brescia|52.172790229648946| 7.223073461460083| 5.439537614166391|  94.267723880597|
|  5 Days|             Sondrio|12.514133626184558|3.5375321378306315|2.5374598791352465|94.86940298507463|
|  5 Days|Monza e della Bri...| 316.5464688474528| 17.79175283235052|12.731670044828006| 94.1231343283582|
|  5 Days|                Lodi| 93.41773258885894| 9.665284920211041|7.0283652874045766|94.72014925373135|
|  5 Days|                Como| 97.27

In [ ]:
result_df_PM10_pd = result_df_PM10.pandas_api()
result_df_PM10_pd

NO2

In [16]:
from pyspark.sql.functions import *
from time import time
import matplotlib.pyplot as plt
import os

data_NO2_prophet = data_NO2.groupBy("NOME", "original_date_time").agg(avg("c_NO2").alias("avg_c_NO2"))
data_NO2_prophet.show()

data_NO2_prophet_u = data_NO2_prophet.withColumnRenamed("avg_c_NO2", "y").withColumnRenamed("original_date_time", "ds")

data_NO2_prophet = data_NO2_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
                                       .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))

data_NO2_prophet.printSchema()

def train_and_forecast(group):
    
    print(f"Processing group: {group['NOME'].iloc[0]} with {len(group)} data points")
    
    group["ds"] = pd.to_datetime(group["ds"])

    m = Prophet(seasonality_mode="additive",
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    interval_width=0.95)
    
    m.fit(group[group['ds'] <= cutoff_date])
    
    forecast = m.predict(group)
    
    path = '/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/NO2/'

    if not os.path.exists(path):
        os.makedirs(path)
    
    try:
        fig1 = m.plot(forecast)
        fig1.suptitle(f"NO2 Concentration Forecast for {group['NOME'].iloc[0]}")
        fig1.legend(['Actual', 'Forecasted'], loc='upper right')
        fig1.subplots_adjust(top=0.9)
        
        fig2 = m.plot_components(forecast)
        fig2.suptitle(f"NO2 Concentration Component Forecast for {group['NOME'].iloc[0]}")
        fig2.subplots_adjust(top=0.9)

        fig1.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/NO2/{group['NOME'].iloc[0]}_prophet_forecast.png")
        fig2.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/NO2/{group['NOME'].iloc[0]}_prophet_components.png")
    except Exception as e:
        print(f"Error generating plots for group {group['NOME'].iloc[0]}: {str(e)}")
    
    
    f_pd = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','trend', 'yearly', 'weekly', 'daily']].set_index("ds")
 
    group_pd = group[['ds', 'NOME', 'y']].set_index("ds")
    
    result_pd = f_pd.join( group_pd, how = "left")
    result_pd.reset_index(level=0, inplace=True)
    
    result_pd['NOME'] = group['NOME'].iloc[0]
    
    return result_pd[['ds', 'NOME', 'y', 'yhat', 'yhat_upper', 'yhat_lower','trend', 'yearly', 'weekly', 'daily']] 

result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('NOME', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat',FloatType()),
    StructField('yhat_upper',FloatType()),
    StructField('yhat_lower',FloatType()),
    StructField('trend',FloatType()),
    StructField('yearly',FloatType()),
    StructField('weekly',FloatType()),
    StructField('daily',FloatType())
  ])

# create train and test data based on cutoff date
cutoff_date = "2021-06-30 23:00:00"

data_NO2_prophet_train = data_NO2_prophet.filter(col('ds') <= cutoff_date)
data_NO2_prophet_test = data_NO2_prophet.filter((col('ds') > cutoff_date))

max_date = data_NO2_prophet_test.agg(max('ds')).collect()[0][0]

period = data_NO2_prophet_test.count()

# Start time
start_time = time()
# Train and forecast by ticker 
spark_forecast_NO2 = data_NO2_prophet.groupBy("NOME").applyInPandas(train_and_forecast, schema=result_schema)
# Processing time
print('The time used for the Spark forecast is ', time()-start_time)

spark_forecast_NO2.show()


+-------+-------------------+------------------+
|   NOME| original_date_time|         avg_c_NO2|
+-------+-------------------+------------------+
|Sondrio|2019-11-24 14:00:00|0.6643196169346735|
| Varese|2020-08-04 06:00:00| 5.275944802631578|
| Varese|2021-07-11 17:00:00|2.3521740331578953|
|Brescia|2019-11-23 07:00:00|3.0555828133666694|
| Varese|2021-06-27 10:00:00| 8.993028231052632|
|  Pavia|2021-06-26 01:00:00| 3.174677009037433|
| Varese|2021-06-26 05:00:00|  4.15398733736842|
| Milano|2021-11-28 12:00:00|18.202434293750002|
| Varese|2022-05-29 00:00:00|2.8546385980263156|
|   Como|2021-05-17 10:00:00| 6.047174076296298|
|Mantova|2019-11-23 22:00:00| 6.472082522758619|
|  Lecco|2022-06-11 19:00:00|3.8186895750980407|
|  Pavia|2022-06-11 03:00:00|4.2664332326203205|
| Varese|2021-06-26 15:00:00| 3.133505573026316|
|Sondrio|2019-11-24 04:00:00|0.4946508185427134|
|   Lodi|2020-08-04 21:00:00| 34.30330264705882|
|Brescia|2021-05-17 12:00:00|2.1098869339400013|
|  Lecco|2022-06-11 

The time used for the Spark forecast is  0.035718679428100586


Processing group: Pavia with 29862 data points                      (0 + 1) / 1]
16:34:13 - cmdstanpy - INFO - Chain [1] start processing
16:34:26 - cmdstanpy - INFO - Chain [1] done processing


+-------------------+-----+------------------+---------+----------+------------+---------+----------+---------+-----------+
|                 ds| NOME|                 y|     yhat|yhat_upper|  yhat_lower|    trend|    yearly|   weekly|      daily|
+-------------------+-----+------------------+---------+----------+------------+---------+----------+---------+-----------+
|2019-01-01 01:00:00|Pavia| 27.90633917647061|30.203426|  45.77675|   13.845307|  9.17825| 12.413368|1.5140492|  7.0977597|
|2019-01-01 02:00:00|Pavia|27.780923018716575|  29.1522| 44.243736|   13.588426|  9.18614|12.4156475|1.5939021|  5.9565115|
|2019-01-01 03:00:00|Pavia|27.680671685561506| 27.91058| 43.589222|   12.057651| 9.194031| 12.417972|1.6574707|   4.641107|
|2019-01-01 04:00:00|Pavia| 26.36605872459893|26.466955| 41.998245|   10.903076|9.2019205|12.4203415|1.7056521|  3.1390407|
|2019-01-01 05:00:00|Pavia|22.540968876470593|24.952797| 40.692352|   7.9132414| 9.209811| 12.422756|1.7394675|  1.5807625|
|2019-01

Processing group: Varese with 29862 data points


In [16]:
from pyspark.sql.functions import col
from pyspark.sql.functions import pow, sqrt
from prophet.diagnostics import performance_metrics


# Calculate MAE
mae_NO2 = spark_forecast_NO2.select(col("NOME"), abs(col("y") - col("yhat")).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MAE")

# Calculate MSE
mse_NO2 = spark_forecast_NO2.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MSE")

# Calculate RMSE
rmse_NO2 = spark_forecast_NO2.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                     .groupBy("NOME") \
                     .agg({"error": "mean"}) \
                     .withColumnRenamed("avg(error)", "MSE") \
                     .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE"))

# Show the results
mae_NO2.show()
mse_NO2.show()
rmse_NO2.show()

23:36:13 - cmdstanpy - INFO - Chain [1] start processing        (40 + 40) / 135]
23:36:26 - cmdstanpy - INFO - Chain [1] done processing             (0 + 6) / 6]
Processing group: Lecco with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Cremona with 29862 data points
23:36:30 - cmdstanpy - INFO - Chain [1] start processing
23:36:30 - cmdstanpy - INFO - Chain [1] start processing
23:36:30 - cmdstanpy - INFO - Chain [1] start processing
23:36:30 - cmdstanpy - INFO - Chain [1] start processing
23:36:30 - cmdstanpy - INFO - Chain [1] start processing
23:36:30 - cmdstanpy - INFO - Chain [1] start processing
23:36:53 - cmdstanpy - INFO - Chain [1] done processing
23:36:56 - cmdstanpy - INFO - Chain [1] done processing
23:36:56 - cmdstanpy - INFO - Chain [1] done processing
23:37:02 - cmdstanpy - INFO - Chain [1] d

+--------------------+------------------+
|                NOME|               MAE|
+--------------------+------------------+
|               Pavia| 6.608959653957249|
|              Varese|12.382239320150621|
|             Brescia| 6.545301868915403|
|             Sondrio| 1.725494214609699|
|Monza e della Bri...| 18.78098509726296|
|                Lodi| 9.170522651473215|
|                Como| 9.686281843633983|
|             Bergamo| 6.585879760774494|
|              Milano|14.475250681124384|
|             Cremona|  7.95202952505299|
|             Mantova| 7.453960377634482|
|               Lecco| 9.049743265319588|
+--------------------+------------------+



Processing group: Lodi with 29862 data points=================> (132 + 3) / 135]
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Bergamo with 29862 data points
23:38:22 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
23:38:22 - cmdstanpy - INFO - Chain [1] start processing
23:38:22 - cmdstanpy - INFO - Chain [1] start processing
23:38:22 - cmdstanpy - INFO - Chain [1] start processing
23:38:22 - cmdstanpy - INFO - Chain [1] start processing
23:38:22 - cmdstanpy - INFO - Chain [1] start processing
23:38:41 - cmdstanpy - INFO - Chain [1] done processing
23:38:42 - cmdstanpy - INFO - Chain [1] done processing
23:38:44 - cmdstanpy - INFO - Chain [1] done processing
23:38:47 - cmdstanpy - INFO - Chain [1] done processing
23:38:47 - cmdstanpy - INFO - Chain [1] done processing
23:38:51 - cmdstanpy - INFO - Ch

+--------------------+------------------+
|                NOME|               MSE|
+--------------------+------------------+
|               Pavia| 76.09514196784025|
|              Varese| 258.4616578257024|
|             Brescia| 76.87624735804437|
|             Sondrio| 6.131850566222194|
|Monza e della Bri...| 620.6790117590781|
|                Lodi|148.25811882979565|
|                Como|162.40994289279703|
|             Bergamo| 75.07395585892348|
|              Milano|349.96949258442726|
|             Cremona|115.38227704396962|
|             Mantova| 101.8089456399404|
|               Lecco|142.73013886357185|
+--------------------+------------------+



Processing group: Bergamo with 29862 data points============>   (127 + 8) / 135]
Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Lecco with 29862 data points
23:40:04 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
23:40:04 - cmdstanpy - INFO - Chain [1] start processing
23:40:04 - cmdstanpy - INFO - Chain [1] start processing
23:40:04 - cmdstanpy - INFO - Chain [1] start processing
23:40:04 - cmdstanpy - INFO - Chain [1] start processing
23:40:05 - cmdstanpy - INFO - Chain [1] start processing
23:40:23 - cmdstanpy - INFO - Chain [1] done processing
23:40:23 - cmdstanpy - INFO - Chain [1] done processing
23:40:27 - cmdstanpy - INFO - Chain [1] done processing
23:40:28 - cmdstanpy - INFO - Chain [1] done processing
23:40:28 - cmdstanpy - INFO - Chain [1] done processing
23:40:33 - cmdstanpy - INFO - Chain

+--------------------+------------------+------------------+
|                NOME|               MSE|              RMSE|
+--------------------+------------------+------------------+
|               Pavia| 76.09514196784025| 8.723252946455252|
|              Varese| 258.4616578257024|16.076742761694682|
|             Brescia| 76.87624735804437| 8.767910090668378|
|             Sondrio| 6.131850566222194|2.4762573707557527|
|Monza e della Bri...| 620.6790117590781|24.913430349092398|
|                Lodi|148.25811882979565|12.176129057701205|
|                Como|162.40994289279703|12.744015964082791|
|             Bergamo| 75.07395585892348|  8.66452282926899|
|              Milano|349.96949258442726| 18.70747157112439|
|             Cremona|115.38227704396962|10.741614266206435|
|             Mantova| 101.8089456399404|10.090041904766323|
|               Lecco|142.73013886357185|11.946971953745093|
+--------------------+------------------+------------------+



In [ ]:
#for long and short term forecast


from pyspark.sql.functions import *
from time import time
import matplotlib.pyplot as plt
import os

data_NO2_prophet = data_NO2.groupBy("NOME", "original_date_time").agg(avg("c_NO2").alias("avg_c_NO2"))
data_NO2_prophet.show()

data_NO2_prophet_u = data_NO2_prophet.withColumnRenamed("avg_c_NO2", "y").withColumnRenamed("original_date_time", "ds")

data_NO2_prophet = data_NO2_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
                                       .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))

data_NO2_prophet.printSchema()

def train_and_forecast(group):
    
    print(f"Processing group: {group['NOME'].iloc[0]} with {len(group)} data points")
    
    group["ds"] = pd.to_datetime(group["ds"])

    m = Prophet(seasonality_mode="additive",
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    interval_width=0.95)
    
    m.fit(group[group['ds'] <= cutoff_date])
    
    forecast = m.predict(group)
    
    path = '/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/NO2/'

    if not os.path.exists(path):
        os.makedirs(path)
    
    #try:
     #   fig1 = m.plot(forecast)
      #  fig1.suptitle(f"NO2 Concentration Forecast for {group['NOME'].iloc[0]}")
       # fig1.legend(['Actual', 'Forecasted'], loc='upper right')
        #fig1.subplots_adjust(top=0.9)
        
        #fig2 = m.plot_components(forecast)
        #fig2.suptitle(f"NO2 Concentration Component Forecast for {group['NOME'].iloc[0]}")
        #fig2.subplots_adjust(top=0.9)

        #fig1.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/NO2/{group['NOME'].iloc[0]}_prophet_forecast.png")
        #fig2.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/NO2/{group['NOME'].iloc[0]}_prophet_components.png")
    #except Exception as e:
     #   print(f"Error generating plots for group {group['NOME'].iloc[0]}: {str(e)}")
    
    
    f_pd = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','trend', 'yearly', 'weekly', 'daily']].set_index("ds")
 
    group_pd = group[['ds', 'NOME', 'y']].set_index("ds")
    
    result_pd = f_pd.join( group_pd, how = "left")
    result_pd.reset_index(level=0, inplace=True)
    
    result_pd['NOME'] = group['NOME'].iloc[0]
    
    return result_pd[['ds', 'NOME', 'y', 'yhat', 'yhat_upper', 'yhat_lower','trend', 'yearly', 'weekly', 'daily']] 

result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('NOME', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat',FloatType()),
    StructField('yhat_upper',FloatType()),
    StructField('yhat_lower',FloatType()),
    StructField('trend',FloatType()),
    StructField('yearly',FloatType()),
    StructField('weekly',FloatType()),
    StructField('daily',FloatType())
  ])

# create train and test data based on cutoff date
cutoff_date = "2021-06-30 23:00:00"

data_NO2_prophet_train = data_NO2_prophet.filter(col('ds') <= cutoff_date)
data_NO2_prophet_test = data_NO2_prophet.filter((col('ds') > cutoff_date))

max_date = data_NO2_prophet_test.agg(max('ds')).collect()[0][0]

period = data_NO2_prophet_test.count()

# Start time
start_time = time()
# Train and forecast by ticker 
spark_forecast_NO2 = data_NO2_prophet.groupBy("NOME").applyInPandas(train_and_forecast, schema=result_schema)
# Processing time
print('The time used for the Spark forecast is ', time()-start_time)

spark_forecast_NO2.show()


In [17]:
from pyspark.sql.functions import col, pow, sqrt

# Define cutoff dates
cutoff_dates = {
    "5 Days": "2021-07-05 23:00:00",
    "30 Days": "2021-07-30 23:00:00",
    "90 Days": "2021-09-28 23:00:00",
    "180 Days": "2021-12-27 23:00:00",
    "365 Days": "2022-06-29 23:00:00"
}

# Define a list to store the evaluation results
results = []

# Compute evaluation metrics for each time period and NOME
for period, cutoff in cutoff_dates.items():
    for nome in spark_forecast_NO2.select("NOME").distinct().rdd.flatMap(lambda x: x).collect():
        # Filter the forecast data based on the cutoff date and NOME
        forecast_data_NO2 = spark_forecast_NO2.filter((col("ds") <= cutoff) & (col("NOME") == nome))

        # Calculate MAE
        mae = forecast_data_NO2.select(abs(col("y") - col("yhat")).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MAE") \
                             .collect()[0]["MAE"]

        # Calculate MSE
        mse = forecast_data_NO2.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MSE") \
                             .collect()[0]["MSE"]

        # Calculate RMSE
        rmse = forecast_data_NO2.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                              .agg({"error": "mean"}) \
                              .withColumnRenamed("avg(error)", "MSE") \
                              .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE")) \
                              .collect()[0]["RMSE"]

        # Calculate coverage
        coverage = forecast_data_NO2.filter((col("y") >= col("yhat_lower")) & (col("y") <= col("yhat_upper"))) \
                                  .count() / forecast_data_NO2.count() * 100

        # Add the results to the list
        results.append((period, nome, mse, rmse, mae, coverage))

# Create a Spark DataFrame from the results list
result_df_NO2 = spark.createDataFrame(results, ["Time", "NOME", "MSE", "RMSE", "MAE", "Coverage"])

# Show the results
result_df_NO2.show(100)

16:34:36 - cmdstanpy - INFO - Chain [1] start processing         (0 + 96) / 135]
16:35:00 - cmdstanpy - INFO - Chain [1] done processing         (96 + 39) / 135]
Processing group: Brescia with 29862 data points                    (0 + 6) / 6]
Processing group: Bergamo with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Pavia with 29862 data points
16:35:13 - cmdstanpy - INFO - Chain [1] start processing
16:35:13 - cmdstanpy - INFO - Chain [1] start processing
16:35:13 - cmdstanpy - INFO - Chain [1] start processing
16:35:13 - cmdstanpy - INFO - Chain [1] start processing
16:35:13 - cmdstanpy - INFO - Chain [1] start processing
16:35:13 - cmdstanpy - INFO - Chain [1] start processing
16:35:28 - cmdstanpy - INFO - Chain [1] done processing
16:35:29 - cmdstanpy - INFO - Chain [1] done processing
16:35:33 - cmdstanpy - INFO - Chain [1] done processing
16:35:34 

16:41:35 - cmdstanpy - INFO - Chain [1] done processing             (4 + 2) / 6]
16:41:44 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Lecco with 29862 data points========>        (113 + 22) / 135]
Processing group: Bergamo with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
16:42:26 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
16:42:26 - cmdstanpy - INFO - Chain [1] start processing
16:42:26 - cmdstanpy - INFO - Chain [1] start processing
16:42:26 - cmdstanpy - INFO - Chain [1] start processing
16:42:26 - cmdstanpy - INFO - Chain [1] start processing
16:42:26 - cmdstanpy - INFO - Chain [1] start processing
16:42:41 - cmdstanpy - INFO - Chain [1] done processing
16:42:41 - cmdstanpy - INFO - Chain [1] done processing
16:42:45 - cmdstanpy - INFO - Chain [1]

16:56:10 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
16:56:26 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Lecco with 29862 data points=======>         (111 + 24) / 135]
Processing group: Cremona with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
16:57:08 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
16:57:08 - cmdstanpy - INFO - Chain [1] start processing
16:57:08 - cmdstanpy - INFO - Chain [1] start processing
16:57:08 - cmdstanpy - INFO - Chain [1] start processing
16:57:08 - cmdstanpy - INFO - Chain [1] start processing
16:57:08 - cmdstanpy - INFO - Chain [1] start processing
16:57:24 - cmdstanpy - INFO - Chain [1] done processing
16:57:26 - cmdstanpy - INFO - Chain [1] done processing
16:57:29 - cmdstanpy - INFO - Chain [1]

17:10:43 - cmdstanpy - INFO - Chain [1] done processing
17:10:47 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
17:11:00 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Pavia with 29862 data points========>        (113 + 22) / 135]
Processing group: Lecco with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lodi with 29862 data points
17:11:46 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
17:11:46 - cmdstanpy - INFO - Chain [1] start processing
17:11:46 - cmdstanpy - INFO - Chain [1] start processing
17:11:46 - cmdstanpy - INFO - Chain [1] start processing
17:11:46 - cmdstanpy - INFO - Chain [1] start processing
17:11:46 - cmdstanpy - INFO - Chain [1] start processing
17:12:02 - cmdstanpy - INFO - Chain [1] done processing
17:12:04 - cmdstanpy - INFO - Chain [1]

17:18:02 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
17:18:05 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
17:18:18 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Pavia with 29862 data points=====>           (104 + 31) / 135]
Processing group: Lecco with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
17:19:02 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
17:19:02 - cmdstanpy - INFO - Chain [1] start processing
17:19:02 - cmdstanpy - INFO - Chain [1] start processing
17:19:02 - cmdstanpy - INFO - Chain [1] start processing
17:19:02 - cmdstanpy - INFO - Chain [1] start processing
17:19:03 - cmdstanpy - INFO - Chain [1] start processing
17:19:17 - cmdstanpy - INFO - Chain [1] done processing
17:19:20 - cmd

...

23:30:15 - cmdstanpy - INFO - Chain [1] start processing
23:30:16 - cmdstanpy - INFO - Chain [1] done processing
23:30:17 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points
23:30:19 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Milano with 29862 data points
Processing group: Monza e della Brianza with 29862 data points
23:30:23 - cmdstanpy - INFO - Chain [1] start processing
23:30:25 - cmdstanpy - INFO - Chain [1] start processing
23:30:30 - cmdstanpy - INFO - Chain [1] done processing
23:30:32 - cmdstanpy - INFO - Chain [1] done processing
23:30:35 - cmdstanpy - INFO - Chain [1] done processing
23:30:38 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
23:30:43 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
23:30:48 - cmdstanpy - INFO - Chain [1] done processing             (4 + 2) / 6]
Processing group: Brescia with 29862 data points========>      (117 + 18) / 135]
Process

23:35:24 - cmdstanpy - INFO - Chain [1] start processing
23:35:24 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points
Processing group: Como with 29862 data points
Processing group: Monza e della Brianza with 29862 data points
23:35:27 - cmdstanpy - INFO - Chain [1] start processing
23:35:27 - cmdstanpy - INFO - Chain [1] start processing
23:35:28 - cmdstanpy - INFO - Chain [1] start processing
23:35:36 - cmdstanpy - INFO - Chain [1] done processing
23:35:42 - cmdstanpy - INFO - Chain [1] done processing
23:35:43 - cmdstanpy - INFO - Chain [1] done processing
23:35:45 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
23:35:45 - cmdstanpy - INFO - Chain [1] done processing
23:35:54 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Lecco with 29862 data points=========>       (114 + 21) / 135]
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data p

23:40:33 - cmdstanpy - INFO - Chain [1] done processing
23:40:33 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points
23:40:36 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Milano with 29862 data points
23:40:40 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Monza e della Brianza with 29862 data points
23:40:42 - cmdstanpy - INFO - Chain [1] start processing
23:40:45 - cmdstanpy - INFO - Chain [1] done processing
23:40:48 - cmdstanpy - INFO - Chain [1] done processing
23:40:51 - cmdstanpy - INFO - Chain [1] done processing
23:40:54 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
23:40:59 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
23:41:02 - cmdstanpy - INFO - Chain [1] done processing             (4 + 2) / 6]
Processing group: Pavia with 29862 data points================> (132 + 3) / 135]
Processing group: Cremona with 29862 data points
Processing grou

Processing group: Sondrio with 29862 data points
23:45:40 - cmdstanpy - INFO - Chain [1] done processing
23:45:41 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points
23:45:44 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Milano with 29862 data points
Processing group: Monza e della Brianza with 29862 data points
23:45:47 - cmdstanpy - INFO - Chain [1] start processing
23:45:48 - cmdstanpy - INFO - Chain [1] done processing
23:45:48 - cmdstanpy - INFO - Chain [1] start processing
23:45:50 - cmdstanpy - INFO - Chain [1] done processing
23:45:57 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
23:46:00 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
23:46:07 - cmdstanpy - INFO - Chain [1] done processing             (3 + 3) / 6]
23:46:21 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Pavia with 29862 data points                     (8 +

+--------+--------------------+------------------+------------------+------------------+-----------------+
|    Time|                NOME|               MSE|              RMSE|               MAE|         Coverage|
+--------+--------------------+------------------+------------------+------------------+-----------------+
|  5 Days|               Pavia| 64.87281671248093| 8.054366313527149| 6.048350489529678| 94.0205223880597|
|  5 Days|              Varese|168.12152310668225| 12.96616840499468|10.039871162846124| 94.2490671641791|
|  5 Days|             Brescia|  39.1891337132023|6.2601224998559175| 4.763917961710172|94.05317164179104|
|  5 Days|             Sondrio|   4.7258483030047|2.1739016314002573| 1.409511416094046|94.96268656716418|
|  5 Days|Monza e della Bri...| 366.8524762902838| 19.15339333617633|14.163134546209132|95.06529850746269|
|  5 Days|                Lodi|111.95185373213585|10.580730302400484| 7.932858644699073|94.44029850746269|
|  5 Days|                Como| 98.44

In [ ]:
result_df_NO2_pd = result_df_NO2.pandas_api()
result_df_NO2_pd

O3 

In [11]:
from pyspark.sql.functions import *
from time import time
import matplotlib.pyplot as plt
import os

data_O3_prophet = data_O3.groupBy("NOME", "original_date_time").agg(avg("c_O3").alias("avg_c_O3"))
data_O3_prophet.show()

data_O3_prophet_u = data_O3_prophet.withColumnRenamed("avg_c_O3", "y").withColumnRenamed("original_date_time", "ds")

data_O3_prophet = data_O3_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
                                       .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))

data_O3_prophet.printSchema()

def train_and_forecast(group):
    
    print(f"Processing group: {group['NOME'].iloc[0]} with {len(group)} data points")
    
    group["ds"] = pd.to_datetime(group["ds"])

    m = Prophet(seasonality_mode="additive",
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    interval_width=0.95)
    
    m.fit(group[group['ds'] <= cutoff_date])
    
    forecast = m.predict(group)
    
    path = '/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/O3/'

    if not os.path.exists(path):
        os.makedirs(path)
    
    try:
        fig1 = m.plot(forecast)
        fig1.suptitle(f"O3 Concentration Forecast for {group['NOME'].iloc[0]}")
        fig1.legend(['Actual', 'Forecasted'], loc='upper right')
        fig1.subplots_adjust(top=0.9)
        
        fig2 = m.plot_components(forecast)
        fig2.suptitle(f"O3 Concentration Component Forecast for {group['NOME'].iloc[0]}")
        fig2.subplots_adjust(top=0.9)

        fig1.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/O3/{group['NOME'].iloc[0]}_prophet_forecast.png")
        fig2.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/O3/{group['NOME'].iloc[0]}_prophet_components.png")
    except Exception as e:
        print(f"Error generating plots for group {group['NOME'].iloc[0]}: {str(e)}")
    
    
    f_pd = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','trend', 'yearly', 'weekly', 'daily']].set_index("ds")
 
    group_pd = group[['ds', 'NOME', 'y']].set_index("ds")
    
    result_pd = f_pd.join( group_pd, how = "left")
    result_pd.reset_index(level=0, inplace=True)
    
    result_pd['NOME'] = group['NOME'].iloc[0]
    
    return result_pd[['ds', 'NOME', 'y', 'yhat', 'yhat_upper', 'yhat_lower','trend', 'yearly', 'weekly', 'daily']] 

result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('NOME', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat',FloatType()),
    StructField('yhat_upper',FloatType()),
    StructField('yhat_lower',FloatType()),
    StructField('trend',FloatType()),
    StructField('yearly',FloatType()),
    StructField('weekly',FloatType()),
    StructField('daily',FloatType())
  ])


# create train and test data based on cutoff date
cutoff_date = "2021-06-30 23:00:00"

data_O3_prophet_train = data_O3_prophet.filter(col('ds') <= cutoff_date)
data_O3_prophet_test = data_O3_prophet.filter((col('ds') > cutoff_date))

max_date = data_O3_prophet_test.agg(max('ds')).collect()[0][0]

period = data_O3_prophet_test.count()

# Start time
start_time = time()
# Train and forecast by ticker 
spark_forecast_O3 = data_O3_prophet.groupBy("NOME").applyInPandas(train_and_forecast, schema=result_schema)
# Processing time
print('The time used for the Spark forecast is ', time()-start_time)

spark_forecast_O3.show()


+-------+-------------------+------------------+
|   NOME| original_date_time|          avg_c_O3|
+-------+-------------------+------------------+
|Sondrio|2019-11-24 14:00:00| 54.41587480904522|
| Varese|2020-08-04 06:00:00|56.921168171052614|
| Varese|2021-07-11 17:00:00|100.52956123684211|
|Brescia|2019-11-23 07:00:00|        54.5038949|
| Varese|2021-06-27 10:00:00|114.93741321052627|
|  Pavia|2021-06-26 01:00:00| 96.90419869518705|
| Varese|2021-06-26 05:00:00| 76.72459271052632|
| Milano|2021-11-28 12:00:00|43.479280416666654|
| Varese|2022-05-29 00:00:00| 78.74115980921054|
|   Como|2021-05-17 10:00:00|112.92797683950621|
|Mantova|2019-11-23 22:00:00| 51.07148930344828|
|  Lecco|2022-06-11 19:00:00| 128.0819811372549|
|  Pavia|2022-06-11 03:00:00|  87.2764229197861|
| Varese|2021-06-26 15:00:00|138.17697368421054|
|Sondrio|2019-11-24 04:00:00| 52.91707371859296|
|   Lodi|2020-08-04 21:00:00|32.878370431372545|
|Brescia|2021-05-17 12:00:00| 107.4413135833333|
|  Lecco|2022-06-11 

The time used for the Spark forecast is  0.04083704948425293


Processing group: Pavia with 29862 data points                      (0 + 1) / 1]
08:19:53 - cmdstanpy - INFO - Chain [1] start processing
08:20:04 - cmdstanpy - INFO - Chain [1] done processing


+-------------------+-----+------------------+---------+----------+-----------+---------+----------+-----------+----------+
|                 ds| NOME|                 y|     yhat|yhat_upper| yhat_lower|    trend|    yearly|     weekly|     daily|
+-------------------+-----+------------------+---------+----------+-----------+---------+----------+-----------+----------+
|2019-01-01 01:00:00|Pavia| 26.25621897379678|18.680555|  46.23206| -10.259371| 62.47475|-30.747225|  -2.127894|-10.919075|
|2019-01-01 02:00:00|Pavia|24.943893596256675|17.960243| 44.659393|  -9.574374|62.460773|-30.740498| -2.0813541|-11.678679|
|2019-01-01 03:00:00|Pavia| 23.92131573650803|17.540604|  45.27844|-10.5055685|  62.4468|-30.733864|  -2.021948|-12.150383|
|2019-01-01 04:00:00|Pavia| 25.58483390833781| 17.29444| 44.771587| -10.240359|62.432823|-30.727325| -1.9508165|-12.460242|
|2019-01-01 05:00:00|Pavia|28.884145283467607|17.139605|   44.9852| -13.005884|62.418846| -30.72088| -1.8691888|-12.689174|
|2019-01

Processing group: Varese with 29862 data points
08:20:12 - cmdstanpy - INFO - Chain [1] start processing                        
08:20:22 - cmdstanpy - INFO - Chain [1] done processing


In [18]:
from pyspark.sql.functions import col
from pyspark.sql.functions import pow, sqrt
from prophet.diagnostics import performance_metrics


# Calculate MAE
mae_O3 = spark_forecast_O3.select(col("NOME"), abs(col("y") - col("yhat")).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MAE")


# Calculate MSE
mse_O3 = spark_forecast_O3.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MSE")

# Calculate RMSE
rmse_O3 = spark_forecast_O3.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                     .groupBy("NOME") \
                     .agg({"error": "mean"}) \
                     .withColumnRenamed("avg(error)", "MSE") \
                     .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE"))

# Show the results
mae_O3.show()
mse_O3.show()
rmse_O3.show()

23:42:47 - cmdstanpy - INFO - Chain [1] start processing        (40 + 40) / 135]
Processing group: Pavia with 29862 data points                      (0 + 6) / 6]
Processing group: Cremona with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Bergamo with 29862 data points
23:43:02 - cmdstanpy - INFO - Chain [1] done processing
23:43:05 - cmdstanpy - INFO - Chain [1] start processing
23:43:05 - cmdstanpy - INFO - Chain [1] start processing
23:43:05 - cmdstanpy - INFO - Chain [1] start processing
23:43:05 - cmdstanpy - INFO - Chain [1] start processing
23:43:05 - cmdstanpy - INFO - Chain [1] start processing
23:43:05 - cmdstanpy - INFO - Chain [1] start processing
23:43:22 - cmdstanpy - INFO - Chain [1] done processing
23:43:31 - cmdstanpy - INFO - Chain [1] done processing
23:43:31 - cmdstanpy - INFO - Chain [1] done processing
23:43:33 - cmdstanpy - INFO - Ch

+--------------------+------------------+
|                NOME|               MAE|
+--------------------+------------------+
|               Pavia| 14.00638484039713|
|              Varese|14.861025223502969|
|             Brescia|10.062276689728158|
|             Sondrio| 8.252120581448573|
|Monza e della Bri...|   16.895872713637|
|                Lodi|15.014291224860811|
|                Como|13.024798574095126|
|             Bergamo|12.133803365008465|
|              Milano|15.273625535837017|
|             Cremona|  13.1068097733092|
|             Mantova|12.159243414757709|
|               Lecco|13.170812347264137|
+--------------------+------------------+



Processing group: Lodi with 29862 data points==================>(133 + 2) / 135]
Processing group: Bergamo with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Pavia with 29862 data points
23:44:47 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
23:44:47 - cmdstanpy - INFO - Chain [1] start processing
23:44:47 - cmdstanpy - INFO - Chain [1] start processing
23:44:47 - cmdstanpy - INFO - Chain [1] start processing
23:44:47 - cmdstanpy - INFO - Chain [1] start processing
23:44:47 - cmdstanpy - INFO - Chain [1] start processing
23:45:07 - cmdstanpy - INFO - Chain [1] done processing
23:45:08 - cmdstanpy - INFO - Chain [1] done processing
23:45:09 - cmdstanpy - INFO - Chain [1] done processing
23:45:10 - cmdstanpy - INFO - Chain [1] done processing
23:45:10 - cmdstanpy - INFO - Chain [1] done processing
23:45:12 - cmdstanpy - INFO - Ch

+--------------------+------------------+
|                NOME|               MSE|
+--------------------+------------------+
|               Pavia|330.38574615151197|
|              Varese| 360.1611801986684|
|             Brescia|163.54788106894082|
|             Sondrio| 114.2234562854702|
|Monza e della Bri...| 495.5008308497702|
|                Lodi| 381.4936251302648|
|                Como| 272.5672819612027|
|             Bergamo| 246.3339952306673|
|              Milano| 390.0958926494945|
|             Cremona| 280.5018336431885|
|             Mantova|237.01368608742655|
|               Lecco|278.43804219664173|
+--------------------+------------------+



Processing group: Lecco with 29862 data points============>    (124 + 11) / 135]
Processing group: Brescia with 29862 data pointsProcessing group: Bergamo with 29862 data points

Processing group: Pavia with 29862 data pointsProcessing group: Cremona with 29862 data points

Processing group: Lodi with 29862 data points
23:46:20 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
23:46:20 - cmdstanpy - INFO - Chain [1] start processing
23:46:20 - cmdstanpy - INFO - Chain [1] start processing
23:46:21 - cmdstanpy - INFO - Chain [1] start processing
23:46:21 - cmdstanpy - INFO - Chain [1] start processing
23:46:21 - cmdstanpy - INFO - Chain [1] start processing
23:46:41 - cmdstanpy - INFO - Chain [1] done processing
23:46:42 - cmdstanpy - INFO - Chain [1] done processing
23:46:43 - cmdstanpy - INFO - Chain [1] done processing
23:46:43 - cmdstanpy - INFO - Chain [1] done processing
23:46:44 - cmdstanpy - INFO - Chain [1] done processing
23:46:47 - cmdstanpy - INFO - Cha

+--------------------+------------------+------------------+
|                NOME|               MSE|              RMSE|
+--------------------+------------------+------------------+
|               Pavia|330.38574615151197| 18.17651633706283|
|              Varese| 360.1611801986684|18.977912956873535|
|             Brescia|163.54788106894082|12.788584013444993|
|             Sondrio| 114.2234562854702| 10.68753742849447|
|Monza e della Bri...| 495.5008308497702|22.259847952081124|
|                Lodi| 381.4936251302648|19.531861793752913|
|                Como| 272.5672819612027|16.509611805284905|
|             Bergamo| 246.3339952306673|15.695030908879005|
|              Milano| 390.0958926494945|19.750845365439286|
|             Cremona| 280.5018336431885| 16.74818896606999|
|             Mantova|237.01368608742655|15.395248815378936|
|               Lecco|278.43804219664173|16.686462842575168|
+--------------------+------------------+------------------+



In [ ]:
#for long and short term forecast


from pyspark.sql.functions import *
from time import time
import matplotlib.pyplot as plt
import os

data_O3_prophet = data_O3.groupBy("NOME", "original_date_time").agg(avg("c_O3").alias("avg_c_O3"))
data_O3_prophet.show()

data_O3_prophet_u = data_O3_prophet.withColumnRenamed("avg_c_O3", "y").withColumnRenamed("original_date_time", "ds")

data_O3_prophet = data_O3_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
                                       .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))

data_O3_prophet.printSchema()

def train_and_forecast(group):
    
    print(f"Processing group: {group['NOME'].iloc[0]} with {len(group)} data points")
    
    group["ds"] = pd.to_datetime(group["ds"])

    m = Prophet(seasonality_mode="additive",
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    interval_width=0.95)
    
    m.fit(group[group['ds'] <= cutoff_date])
    
    forecast = m.predict(group)
    
    path = '/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/O3/'

    if not os.path.exists(path):
        os.makedirs(path)
    
   # try:
    #    fig1 = m.plot(forecast)
     #   fig1.suptitle(f"O3 Concentration Forecast for {group['NOME'].iloc[0]}")
     #   fig1.legend(['Actual', 'Forecasted'], loc='upper right')
      #  fig1.subplots_adjust(top=0.9)
        
       # fig2 = m.plot_components(forecast)
       # fig2.suptitle(f"O3 Concentration Component Forecast for {group['NOME'].iloc[0]}")
        #fig2.subplots_adjust(top=0.9)

       # fig1.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/O3/{group['NOME'].iloc[0]}_prophet_forecast.png")
        #fig2.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/O3/{group['NOME'].iloc[0]}_prophet_components.png")
    #except Exception as e:
     #   print(f"Error generating plots for group {group['NOME'].iloc[0]}: {str(e)}")
    
    
    f_pd = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','trend', 'yearly', 'weekly', 'daily']].set_index("ds")
 
    group_pd = group[['ds', 'NOME', 'y']].set_index("ds")
    
    result_pd = f_pd.join( group_pd, how = "left")
    result_pd.reset_index(level=0, inplace=True)
    
    result_pd['NOME'] = group['NOME'].iloc[0]
    
    return result_pd[['ds', 'NOME', 'y', 'yhat', 'yhat_upper', 'yhat_lower','trend', 'yearly', 'weekly', 'daily']] 

result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('NOME', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat',FloatType()),
    StructField('yhat_upper',FloatType()),
    StructField('yhat_lower',FloatType()),
    StructField('trend',FloatType()),
    StructField('yearly',FloatType()),
    StructField('weekly',FloatType()),
    StructField('daily',FloatType())
  ])


# create train and test data based on cutoff date
cutoff_date = "2021-06-30 23:00:00"

data_O3_prophet_train = data_O3_prophet.filter(col('ds') <= cutoff_date)
data_O3_prophet_test = data_O3_prophet.filter((col('ds') > cutoff_date))

max_date = data_O3_prophet_test.agg(max('ds')).collect()[0][0]

period = data_O3_prophet_test.count()

# Start time
start_time = time()
# Train and forecast by ticker 
spark_forecast_O3 = data_O3_prophet.groupBy("NOME").applyInPandas(train_and_forecast, schema=result_schema)
# Processing time
print('The time used for the Spark forecast is ', time()-start_time)

spark_forecast_O3.show()


In [12]:
from pyspark.sql.functions import col, pow, sqrt

# Define cutoff dates
cutoff_dates = {
    "5 Days": "2021-07-05 23:00:00",
    "30 Days": "2021-07-30 23:00:00",
    "90 Days": "2021-09-28 23:00:00",
    "180 Days": "2021-12-27 23:00:00",
    "365 Days": "2022-06-29 23:00:00"
}

# Define a list to store the evaluation results
results = []

# Compute evaluation metrics for each time period and NOME
for period, cutoff in cutoff_dates.items():
    for nome in spark_forecast_O3.select("NOME").distinct().rdd.flatMap(lambda x: x).collect():
        # Filter the forecast data based on the cutoff date and NOME
        forecast_data_O3 = spark_forecast_O3.filter((col("ds") <= cutoff) & (col("NOME") == nome))

        # Calculate MAE
        mae = forecast_data_O3.select(abs(col("y") - col("yhat")).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MAE") \
                             .collect()[0]["MAE"]

        # Calculate MSE
        mse = forecast_data_O3.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MSE") \
                             .collect()[0]["MSE"]

        # Calculate RMSE
        rmse = forecast_data_O3.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                              .agg({"error": "mean"}) \
                              .withColumnRenamed("avg(error)", "MSE") \
                              .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE")) \
                              .collect()[0]["RMSE"]

        # Calculate coverage
        coverage = forecast_data_O3.filter((col("y") >= col("yhat_lower")) & (col("y") <= col("yhat_upper"))) \
                                  .count() / forecast_data_O3.count() * 100

        # Add the results to the list
        results.append((period, nome, mse, rmse, mae, coverage))

# Create a Spark DataFrame from the results list
result_df_O3 = spark.createDataFrame(results, ["Time", "NOME", "MSE", "RMSE", "MAE", "Coverage"])

# Show the results
result_df_O3.show()


Processing group: Bergamo with 29862 data points                    (0 + 6) / 6]
Processing group: Lecco with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
08:20:53 - cmdstanpy - INFO - Chain [1] start processing
08:20:53 - cmdstanpy - INFO - Chain [1] start processing
08:20:53 - cmdstanpy - INFO - Chain [1] start processing
08:20:53 - cmdstanpy - INFO - Chain [1] start processing
08:20:53 - cmdstanpy - INFO - Chain [1] start processing
08:20:53 - cmdstanpy - INFO - Chain [1] start processing
08:21:09 - cmdstanpy - INFO - Chain [1] done processing
08:21:10 - cmdstanpy - INFO - Chain [1] done processing
08:21:11 - cmdstanpy - INFO - Chain [1] done processing
08:21:11 - cmdstanpy - INFO - Chain [1] done processing
08:21:12 - cmdstanpy - INFO - Chain [1] done processing
08:21:13 - cmdstanpy - INFO - Chain [1] done processing
Pro

08:31:19 - cmdstanpy - INFO - Chain [1] done processing             (3 + 3) / 6]
08:31:22 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Lecco with 29862 data points                     (7 + 7) / 14]
Processing group: Cremona with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Bergamo with 29862 data points
08:31:57 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:31:57 - cmdstanpy - INFO - Chain [1] start processing
08:31:57 - cmdstanpy - INFO - Chain [1] start processing
08:31:57 - cmdstanpy - INFO - Chain [1] start processing
08:31:57 - cmdstanpy - INFO - Chain [1] start processing
08:31:57 - cmdstanpy - INFO - Chain [1] start processing
08:32:12 - cmdstanpy - INFO - Chain [1] done processing
08:32:13 - cmdstanpy - INFO - Chain [1] done processing
08:32:13 - cmdstanpy - INFO - Chain [1] done processing
08:32:14

08:42:11 - cmdstanpy - INFO - Chain [1] done processing
08:42:14 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
08:42:19 - cmdstanpy - INFO - Chain [1] done processing             (4 + 2) / 6]
Processing group: Pavia with 29862 data points================>   (13 + 1) / 14]
Processing group: Brescia with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Lodi with 29862 data points
08:42:51 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:42:51 - cmdstanpy - INFO - Chain [1] start processing
08:42:51 - cmdstanpy - INFO - Chain [1] start processing
08:42:51 - cmdstanpy - INFO - Chain [1] start processing
08:42:51 - cmdstanpy - INFO - Chain [1] start processing
08:42:51 - cmdstanpy - INFO - Chain [1] start processing
08:43:07 - cmdstanpy - INFO - Chain [1] done processing
08:43:07 - cmdstanpy - INFO - Chain [1]

08:47:23 - cmdstanpy - INFO - Chain [1] done processing
08:47:23 - cmdstanpy - INFO - Chain [1] done processing
08:47:31 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Lecco with 29862 data points===>             (100 + 35) / 135]
Processing group: Lodi with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Brescia with 29862 data points
08:48:01 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:48:01 - cmdstanpy - INFO - Chain [1] start processing
08:48:01 - cmdstanpy - INFO - Chain [1] start processing
08:48:01 - cmdstanpy - INFO - Chain [1] start processing
08:48:01 - cmdstanpy - INFO - Chain [1] start processing
08:48:01 - cmdstanpy - INFO - Chain [1] start processing
08:48:16 - cmdstanpy - INFO - Chain [1] done processing
08:48:17 - cmdstanpy - INFO - Chain [1] done processing
08:48:18

08:52:44 - cmdstanpy - INFO - Chain [1] done processing
08:52:46 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
08:52:49 - cmdstanpy - INFO - Chain [1] done processing             (3 + 3) / 6]
Processing group: Lecco with 29862 data points======>          (107 + 28) / 135]
Processing group: Bergamo with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
08:53:19 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:53:19 - cmdstanpy - INFO - Chain [1] start processing
08:53:19 - cmdstanpy - INFO - Chain [1] start processing
08:53:19 - cmdstanpy - INFO - Chain [1] start processing
08:53:19 - cmdstanpy - INFO - Chain [1] start processing
08:53:19 - cmdstanpy - INFO - Chain [1] start processing
08:53:33 - cmdstanpy - INFO - Chain [1] done processing
08:53:36 - cmdstanpy - INFO - Chain [1]

...

15:23:14 - cmdstanpy - INFO - Chain [1] done processing
15:23:14 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
15:23:23 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Cremona with 29862 data points==============>   (13 + 1) / 14]
Processing group: Bergamo with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Lecco with 29862 data points
15:24:06 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
15:24:06 - cmdstanpy - INFO - Chain [1] start processing
15:24:06 - cmdstanpy - INFO - Chain [1] start processing
15:24:06 - cmdstanpy - INFO - Chain [1] start processing
15:24:06 - cmdstanpy - INFO - Chain [1] start processing
15:24:06 - cmdstanpy - INFO - Chain [1] start processing
15:24:22 - cmdstanpy - INFO - Chain [1] done processing
15:24:24 - cmdstanpy - INFO - Chain [1] d

15:37:04 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
15:37:12 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Lecco with 29862 data points===>             (100 + 35) / 135]
Processing group: Brescia with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Bergamo with 29862 data points
15:37:56 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
15:37:56 - cmdstanpy - INFO - Chain [1] start processing
15:37:56 - cmdstanpy - INFO - Chain [1] start processing
15:37:57 - cmdstanpy - INFO - Chain [1] start processing
15:37:57 - cmdstanpy - INFO - Chain [1] start processing
15:37:57 - cmdstanpy - INFO - Chain [1] start processing
15:38:15 - cmdstanpy - INFO - Chain [1] done processing
15:38:16 - cmdstanpy - INFO - Chain [1] done processing
15:38:18 - cmdstanpy - INFO - Chain [1]

15:50:42 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
15:50:44 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
Processing group: Cremona with 29862 data points======>           (11 + 3) / 14]
Processing group: Lecco with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
15:51:29 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
15:51:29 - cmdstanpy - INFO - Chain [1] start processing
15:51:29 - cmdstanpy - INFO - Chain [1] start processing
15:51:29 - cmdstanpy - INFO - Chain [1] start processing
15:51:29 - cmdstanpy - INFO - Chain [1] start processing
15:51:30 - cmdstanpy - INFO - Chain [1] start processing
15:51:46 - cmdstanpy - INFO - Chain [1] done processing
15:51:49 - cmdstanpy - INFO - Chain [1] done processing
15:51:49 - cmdstanpy - INFO - Chain [1] d

15:57:28 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
15:57:33 - cmdstanpy - INFO - Chain [1] done processing             (3 + 3) / 6]
Processing group: Pavia with 29862 data points============>    (124 + 11) / 135]
Processing group: Cremona with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Brescia with 29862 data points
15:58:16 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
15:58:16 - cmdstanpy - INFO - Chain [1] start processing
15:58:16 - cmdstanpy - INFO - Chain [1] start processing
15:58:16 - cmdstanpy - INFO - Chain [1] start processing
15:58:16 - cmdstanpy - INFO - Chain [1] start processing
15:58:16 - cmdstanpy - INFO - Chain [1] start processing
15:58:36 - cmdstanpy - INFO - Chain [1] done processing
15:58:36 - cmdstanpy - INFO - Chain [1] done processing
15:58:37 - cmdstanpy - INFO - Chain [1]

+-------+--------------------+------------------+------------------+------------------+-----------------+
|   Time|                NOME|               MSE|              RMSE|               MAE|         Coverage|
+-------+--------------------+------------------+------------------+------------------+-----------------+
| 5 Days|               Pavia|203.36154505266364|14.260488948583202|11.195181543552085|94.83208955223881|
| 5 Days|              Varese|  284.033452997258| 16.85329205221514|13.133191875256578|95.20522388059702|
| 5 Days|             Brescia|130.19006004281252|11.410085891123368| 9.013008861723712|94.73880597014926|
| 5 Days|             Sondrio| 76.15051413007892| 8.726426194615922| 6.878374133794683|94.71548507462687|
| 5 Days|Monza e della Bri...|375.62287271620784|19.380992562719996|14.830862005826122|94.84608208955224|
| 5 Days|                Lodi| 239.7261024010531|15.483090854253007|12.072230496388475|94.99067164179105|
| 5 Days|                Como|226.286991094259

In [15]:
result_df_O3_pd = result_df_O3.pandas_api()
result_df_O3_pd

,Time,NOME,MSE,RMSE,MAE,Coverage
0,5 Days,Pavia,203.361545,14.260489,11.195182,94.832090
1,5 Days,Varese,284.033453,16.853292,13.133192,95.205224
2,5 Days,Brescia,130.190060,11.410086,9.013009,94.738806
3,5 Days,Sondrio,76.150514,8.726426,6.878374,94.715485
4,5 Days,Monza e della Brianza,375.622873,19.380993,14.830862,94.846082
5,5 Days,Lodi,239.726102,15.483091,12.072230,94.990672
6,5 Days,Como,226.286991,15.042839,11.926102,94.832090
7,5 Days,Bergamo,159.098424,12.613422,9.890054,94.589552
8,5 Days,Milano,301.256803,17.356751,13.364895,95.452425
9,5 Days,Cremona,203.556146,14.267310,11.204218,94.776119
